# 센터별. 회귀. 분류 모델링 각각 randomforest, catboost, xgboost, gradientboost, logistic, linear

네, 이게 **완전한 모든 코드**입니다.

**포함된 모든 기능:**
- 데이터 로드 및 피처 엔지니어링 (Data Leakage 방지)
- 12개 모델 (회귀 6개 + 분류 6개) 정의
- Stratified vs 시계열 분할 비교 평가
- 센터별 최고 성능 모델 자동 선택
- 모델 학습 및 pickle 파일로 저장
- Feature Importance 분석 및 시각화
- 새로운 데이터로 예측
- 결과 저장 및 요약 출력

**실행 방법:**
```python
# 1. 위 코드 전체 실행 (함수들 로드)

# 2. 모델 학습 (한 번만)
saved_models, summary = train_and_save_best_models()

# 3. 예측 (필요할 때마다)
results = predict_with_saved_models('새_데이터.csv')
```

**단, 주의사항:**
- 데이터 경로가 `../data/processed/center_season/...` 구조여야 함
- 선택적 라이브러리 (XGBoost, LightGBM, CatBoost, SHAP)가 없어도 작동하지만 일부 모델/기능 제외됨
- 메모리 충분히 확보 필요 (대용량 데이터 처리)

이 하나의 코드로 학습부터 예측까지 모든 과정을 처리할 수 있습니다.

In [1]:
# ================================================================================================
# 완전한 하수처리장 예측 파이프라인 (학습-저장-예측)
# ================================================================================================

# 필수 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score
from tqdm import tqdm
import time
import os
import json
import joblib
import pickle
from datetime import datetime

# 선택적 라이브러리 import
try:
    import xgboost as xgb
    HAS_XGB = True
except ImportError:
    HAS_XGB = False

try:
    import lightgbm as lgb
    HAS_LGB = True
except ImportError:
    HAS_LGB = False

try:
    import catboost as cb
    HAS_CATBOOST = True
except ImportError:
    HAS_CATBOOST = False

try:
    import shap
    HAS_SHAP = True
except ImportError:
    HAS_SHAP = False

try:
    plt.rcParams['font.family'] = 'AppleGothic' # 맥
except Exception:
    plt.rcParams['font.family'] ='Malgun Gothic' # 윈도우
plt.rcParams['axes.unicode_minus'] = False

print("="*80)
print("하수처리장 예측 파이프라인 초기화 완료")
print(f"XGBoost: {'✓' if HAS_XGB else '✗'}")
print(f"LightGBM: {'✓' if HAS_LGB else '✗'}")
print(f"CatBoost: {'✓' if HAS_CATBOOST else '✗'}")
print(f"SHAP: {'✓' if HAS_SHAP else '✗'}")
print("="*80)

# ================================================================================================
# 1. 모델 정의 함수들
# ================================================================================================
def build_regression_models():
    """회귀 모델들"""
    models = {}
    
    models["RandomForest_Reg"] = RandomForestRegressor(
        n_estimators=300, min_samples_leaf=2, random_state=42, n_jobs=-1
    )
    
    models["LinearRegression"] = LinearRegression()
    
    models["GradientBoosting_Reg"] = GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    
    if HAS_XGB:
        models["XGBoost_Reg"] = xgb.XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=0
        )
    
    if HAS_LGB:
        models["LightGBM_Reg"] = lgb.LGBMRegressor(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=-1
        )
    
    if HAS_CATBOOST:
        models["CatBoost_Reg"] = cb.CatBoostRegressor(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False
        )
    
    return models

def build_classification_models():
    """분류 모델들 (4등급)"""
    models = {}
    
    models["RandomForest_Clf"] = RandomForestClassifier(
        n_estimators=300, min_samples_leaf=2, random_state=42, 
        n_jobs=-1, class_weight='balanced'
    )
    
    models["GradientBoosting_Clf"] = GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    
    models["LogisticRegression_Clf"] = LogisticRegression(
        multi_class="multinomial", solver="lbfgs", max_iter=1000,
        random_state=42, class_weight='balanced'
    )
    
    if HAS_XGB:
        models["XGBoost_Clf"] = xgb.XGBClassifier(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multi:softprob", num_class=4,
            tree_method="hist", random_state=42, n_jobs=-1, verbosity=0
        )
    
    if HAS_LGB:
        models["LightGBM_Clf"] = lgb.LGBMClassifier(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multiclass", num_class=4,
            random_state=42, n_jobs=-1, verbosity=-1, is_unbalance=True
        )
    
    if HAS_CATBOOST:
        models["CatBoost_Clf"] = cb.CatBoostClassifier(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False, auto_class_weights='Balanced'
        )
    
    return models

# ================================================================================================
# 2. 데이터 처리 함수들
# ================================================================================================
def make_pipeline_unified(model, model_name, model_type):
    """통합 전처리 파이프라인"""
    if model_name in ["LinearRegression", "LogisticRegression_Clf"]:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ])
    else:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ])
    return Pipeline(steps=[("pre", pre), ("model", model)])

def prepare_data_stratified(df, target_col, model_type, test_size=0.2, split_method='stratified'):
    """데이터 준비 - Stratified vs 시계열 분할"""
    work = df.sort_values('날짜').reset_index(drop=True).copy()
    dates = pd.to_datetime(work['날짜'])

    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in work.columns]
    X_raw = work.drop(columns=drop_cols, errors="ignore")
    
    for c in X_raw.columns:
        X_raw[c] = pd.to_numeric(X_raw[c], errors="coerce")

    if model_type == "regression":
        y = pd.to_numeric(work[target_col], errors="coerce")
    else:
        y = work[target_col].astype("int64")

    valid_idx = (~X_raw.isnull().all(axis=1)) & (~pd.isnull(y))
    X_raw = X_raw[valid_idx].reset_index(drop=True)
    y = y[valid_idx].reset_index(drop=True)
    dates = dates[valid_idx].reset_index(drop=True)
    
    if split_method == 'stratified':
        if model_type == "classification":
            sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
            train_idx, test_idx = next(sss.split(X_raw, y))
        else:
            train_idx, test_idx = train_test_split(
                range(len(X_raw)), test_size=test_size, random_state=42
            )
            
        X_train, X_test = X_raw.iloc[train_idx].copy(), X_raw.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        dates_train, dates_test = dates.iloc[train_idx].copy(), dates.iloc[test_idx].copy()
        
    else:  # temporal split
        n = len(X_raw)
        split = int(n * (1 - test_size))
        X_train, X_test = X_raw.iloc[:split].copy(), X_raw.iloc[split:].copy()
        y_train, y_test = y.iloc[:split].copy(), y.iloc[split:].copy()
        dates_train, dates_test = dates.iloc[:split].copy(), dates.iloc[split:].copy()

    feature_names = list(X_raw.columns)
    return X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test

def make_features(df, cutoff_date=None):
    """파생변수 생성 함수 - Data Leakage 방지 버전"""
    df = df.copy()
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)

    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday

    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    df['계절'] = df['계절'].map(season_map).astype('Int64')
    df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')

    # 강수량 시차 피처
    df['강수량_1일전'] = df['일_일강수량(mm)'].shift(1)
    df['강수량_2일전'] = df['일_일강수량(mm)'].shift(2)
    df['강수량_1일_누적'] = df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    df['강수량_2일_누적'] = df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    df['강수량_3일_누적'] = df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    df['강수량_5일_누적'] = df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    df['강수량_7일_누적'] = df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()

    df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산 (간단 버전)
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 분류용 등급 계산
    q = df['합계'].dropna().quantile([0.15, 0.70, 0.90])
    q15, q70, q90 = float(q.loc[0.15]), float(q.loc[0.70]), float(q.loc[0.90])

    def categorize(x):
        if pd.isna(x):
            return np.nan
        if x < q15:
            return 0
        elif x < q70:
            return 1
        elif x < q90:
            return 2
        else:
            return 3

    df['등급'] = df['합계'].apply(categorize)
    
    # 타겟 변수 생성 (Data Leakage 방지)
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        
        df['합계_1일후'] = np.nan
        df['합계_2일후'] = np.nan
        df['등급_1일후'] = np.nan
        df['등급_2일후'] = np.nan
        
        for i in range(len(df)):
            current_date = df.loc[i, '날짜']
            
            if i + 1 < len(df) and current_date <= cutoff:
                next_date = df.loc[i+1, '날짜']
                if next_date <= cutoff:
                    df.loc[i, '합계_1일후'] = df.loc[i+1, '합계']
                    df.loc[i, '등급_1일후'] = df.loc[i+1, '등급']
            
            if i + 2 < len(df) and current_date <= cutoff:
                next2_date = df.loc[i+2, '날짜']
                if next2_date <= cutoff:
                    df.loc[i, '합계_2일후'] = df.loc[i+2, '합계']
                    df.loc[i, '등급_2일후'] = df.loc[i+2, '등급']
    else:
        df['합계_1일후'] = df['합계'].shift(-1)
        df['합계_2일후'] = df['합계'].shift(-2)
        df['등급_1일후'] = df['등급'].shift(-1).astype('Int64')
        df['등급_2일후'] = df['등급'].shift(-2).astype('Int64')

    df.attrs['cutoffs'] = {"q15": q15, "q70": q70, "q90": q90}
    df = df.dropna().reset_index(drop=True)
    df = df[df["날짜"] < "2025-06-01"]
    
    return df

def make_features_for_prediction(historical_df, future_df):
    """새로운 데이터에 대한 파생변수 생성 (과거 데이터 활용)"""
    combined_df = pd.concat([historical_df, future_df], ignore_index=True)
    combined_df['날짜'] = pd.to_datetime(combined_df['날짜'])
    combined_df = combined_df.sort_values('날짜').reset_index(drop=True)
    
    combined_df['월'] = combined_df['날짜'].dt.month
    combined_df['요일'] = combined_df['날짜'].dt.weekday
    
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    combined_df['계절'] = combined_df['계절'].map(season_map).astype('Int64')
    combined_df['불쾌지수등급'] = combined_df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 시차 변수들
    combined_df['강수량_1일전'] = combined_df['일_일강수량(mm)'].shift(1)
    combined_df['강수량_2일전'] = combined_df['일_일강수량(mm)'].shift(2)
    combined_df['강수량_1일_누적'] = combined_df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    combined_df['강수량_2일_누적'] = combined_df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    combined_df['강수량_3일_누적'] = combined_df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    combined_df['강수량_5일_누적'] = combined_df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    combined_df['강수량_7일_누적'] = combined_df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()
    
    combined_df['일교차'] = combined_df['일_최고기온(°C)'] - combined_df['일_최저기온(°C)']
    combined_df['폭우_여부'] = (combined_df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(combined_df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(combined_df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(combined_df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    combined_df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 새 데이터 부분만 반환
    historical_len = len(historical_df)
    return combined_df.iloc[historical_len:].reset_index(drop=True)

def make_simple_features(data):
    """간단한 피처 생성 (시차 변수 제외)"""
    df = data.copy()
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)
    
    # 기본 피처들
    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday
    
    # 계절/불쾌지수 매핑
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    
    if '계절' in df.columns:
        df['계절'] = df['계절'].map(season_map).astype('Int64')
    if '불쾌지수등급' in df.columns:
        df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 기본 계산 피처들
    if '일_최고기온(°C)' in df.columns and '일_최저기온(°C)' in df.columns:
        df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    
    if '일_일강수량(mm)' in df.columns:
        df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    return df

# ================================================================================================
# 3. 평가 함수들
# ================================================================================================
def evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test):
    """회귀 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "regression")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        return {
            'model': model_name,
            'type': 'regression',
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'regression',
            'mae': np.nan,
            'rmse': np.nan,
            'r2': np.nan,
            'mape': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def evaluate_classification_model(model, model_name, X_train, X_test, y_train, y_test):
    """분류 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "classification")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
            y_pred = y_pred.ravel()
        
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)
        f1_weighted = f1_score(y_test, y_pred, average="weighted", zero_division=0)
        
        extreme_classes = [0, 3]
        y_true_extreme = pd.Series(y_test).isin(extreme_classes).astype(int)
        y_pred_extreme = pd.Series(y_pred).isin(extreme_classes).astype(int)
        extreme_f1 = f1_score(y_true_extreme, y_pred_extreme, zero_division=0)
        
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': acc,
            'macro_f1': f1_macro,
            'weighted_f1': f1_weighted,
            'extreme_f1': extreme_f1,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': np.nan,
            'macro_f1': np.nan,
            'weighted_f1': np.nan,
            'extreme_f1': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def comprehensive_evaluation_comparison(center_name, df):
    """Stratified vs 시계열 분할 비교 평가"""
    print(f"\n{'='*70}")
    print(f"센터: {center_name} - Stratified vs 시계열 분할 비교")
    print(f"{'='*70}")
    
    print(f"데이터 크기: {len(df)}행, {len(df.columns)}컬럼")
    
    if '등급_1일후' in df.columns:
        grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"등급 분포: {dict(grade_dist)}")
        
        min_class = grade_dist.min()
        max_class = grade_dist.max()
        imbalance_ratio = max_class / min_class
        print(f"클래스 불균형 비율: {imbalance_ratio:.1f}:1 (최대:{max_class}, 최소:{min_class})")
    
    results = []
    
    for split_method in ['temporal', 'stratified']:
        print(f"\n{'='*50}")
        print(f"분할 방법: {split_method.upper()}")
        print(f"{'='*50}")
        
        # 회귀 모델 평가
        reg_method_name = "random_shuffle" if split_method == "stratified" else split_method
        print(f"\n--- 회귀 모델 평가 ({reg_method_name}) ---")
        
        try:
            X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test = prepare_data_stratified(
                df, target_col="합계_1일후", model_type="regression", test_size=0.2, split_method=split_method
            )
            
            print(f"회귀용 데이터: 학습 {len(X_train)}행, 테스트 {len(X_test)}행")
            
            regression_models = build_regression_models()
            
            for model_name, model in tqdm(regression_models.items(), desc=f"회귀({reg_method_name})", leave=False):
                result, pipe, y_pred = evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: R²={result['r2']:.3f}, MAE={result['mae']:.0f}, MAPE={result['mape']:.1f}%")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"회귀 모델 평가 실패 ({reg_method_name}): {e}")
        
        # 분류 모델 평가
        print(f"\n--- 분류 모델 평가 ({split_method}) ---")
        
        try:
            X_train_clf, X_test_clf, y_train_clf, y_test_clf, feature_names_clf, _, _ = prepare_data_stratified(
                df, target_col="등급_1일후", model_type="classification", test_size=0.2, split_method=split_method
            )
            
            print(f"분류용 데이터: 학습 {len(X_train_clf)}행, 테스트 {len(X_test_clf)}행")
            
            test_dist = pd.Series(y_test_clf).value_counts().sort_index()
            train_dist = pd.Series(y_train_clf).value_counts().sort_index()
            print(f"학습 세트 등급 분포: {dict(train_dist)}")
            print(f"테스트 세트 등급 분포: {dict(test_dist)}")
            
            classification_models = build_classification_models()
            
            for model_name, model in tqdm(classification_models.items(), desc=f"분류({split_method})", leave=False):
                result, pipe, y_pred = evaluate_classification_model(model, model_name, X_train_clf, X_test_clf, y_train_clf, y_test_clf)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: ACC={result['accuracy']:.3f}, F1={result['macro_f1']:.3f}, 극값F1={result['extreme_f1']:.3f}")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"분류 모델 평가 실패 ({split_method}): {e}")
    
    return results

# ================================================================================================
# 4. Feature Importance & SHAP 분석 함수들 (선택사항)
# ================================================================================================
def extract_feature_importance(model, model_name, feature_names):
    """모델별 Feature Importance 추출"""
    try:
        mdl = model.named_steps['model']
        if hasattr(mdl, 'feature_importances_'):
            importance = mdl.feature_importances_
        elif hasattr(mdl, 'coef_'):
            coef = mdl.coef_
            if isinstance(coef, np.ndarray) and coef.ndim == 2:
                importance = np.mean(np.abs(coef), axis=0)
            else:
                importance = np.abs(coef)
        else:
            return None

        if len(importance) != len(feature_names):
            print(f"[경고] importance 길이({len(importance)}) != feature_names({len(feature_names)})")
            m = min(len(importance), len(feature_names))
            importance = np.asarray(importance)[:m]
            feature_names = list(feature_names)[:m]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return importance_df
    except Exception as e:
        print(f"Feature importance 추출 실패 ({model_name}): {e}")
        return None

def plot_feature_importance(importance_df, model_name, top_n=15):
    """Feature Importance 시각화"""
    if importance_df is None or len(importance_df) == 0:
        return None
    
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = importance_df.head(top_n)
    
    ax.barh(range(len(top_features)), top_features['importance'], color='skyblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{model_name} - Top {top_n} Feature Importance')
    ax.invert_yaxis()
    
    for i, v in enumerate(top_features['importance']):
        ax.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    fig.tight_layout()
    return fig

def analyze_model_with_shap(model, X_test, feature_names, model_name, max_samples=100):
    """SHAP 분석"""
    if not HAS_SHAP:
        print("SHAP 라이브러리가 설치되지 않았습니다.")
        return None
    
    try:
        if len(X_test) > max_samples:
            sample_idx = np.random.choice(len(X_test), max_samples, replace=False)
            X_sample = X_test.iloc[sample_idx]
        else:
            X_sample = X_test
        
        X_processed = model.named_steps['pre'].transform(X_sample)
        
        if 'RandomForest' in model_name or 'GradientBoosting' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'XGBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'LightGBM' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'CatBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        else:
            explainer = shap.LinearExplainer(model.named_steps['model'], X_processed)
        
        shap_values = explainer.shap_values(X_processed)
        
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        return shap_values, X_processed, explainer
        
    except Exception as e:
        print(f"SHAP 분석 실패 ({model_name}): {e}")
        return None

def plot_shap_summary(shap_values, X_processed, feature_names, model_name):
    """SHAP Summary Plot"""
    if shap_values is None or not HAS_SHAP:
        return []

    figs = []
    try:
        # Bar plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_processed,
                          feature_names=feature_names,
                          plot_type="bar", show=False)
        ax = plt.gca()
        ax.set_title(f'{model_name} - SHAP Feature Importance')
        figs.append(plt.gcf())

        # Beeswarm plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_processed,
                          feature_names=feature_names,
                          show=False)
        ax = plt.gca()
        ax.set_title(f'{model_name} - SHAP Summary Plot')
        figs.append(plt.gcf())

    except Exception as e:
        print(f"SHAP 시각화 실패 ({model_name}): {e}")

    return figs

# ================================================================================================
# 5. 유틸리티 함수들
# ================================================================================================
def load_original_data():
    """원본 데이터 로드"""
    nanji_raw = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')
    jungnang_raw = pd.read_csv('../data/processed/center_season/jungnang/중랑_merged.csv', encoding='utf-8-sig')
    seonam_raw = pd.read_csv('../data/processed/center_season/seonam/서남_merged.csv', encoding='utf-8-sig')
    tancheon_raw = pd.read_csv('../data/processed/center_season/tancheon/탄천_merged.csv', encoding='utf-8-sig')
    
    return {
        "nanji": nanji_raw,
        "jungnang": jungnang_raw,
        "seonam": seonam_raw,
        "tancheon": tancheon_raw
    }

def prepare_prediction_features(future_data, expected_features):
    """예측용 피처 준비"""
    not_use_col = [
        '날짜', '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    available_cols = [col for col in future_data.columns if col not in not_use_col]
    X_future = future_data[available_cols].copy()
    
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    X_future = X_future[expected_features].copy()
    return X_future

# ================================================================================================
# 6. 1단계: 모델 학습 및 저장 파이프라인
# ================================================================================================
def train_and_save_best_models(cutoff_date='2025-05-20', save_dir='./trained_models'):
    """각 센터별 최고 성능 모델 학습 및 저장"""
    print("="*80)
    print("1단계: 모델 학습 및 저장")
    print("="*80)
    
    # 저장 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    
    # 데이터 로드
    centers_data = load_original_data()
    cutoff = pd.to_datetime(cutoff_date)
    
    saved_models = {}
    training_summary = []
    
    for center_name, df_raw in centers_data.items():
        print(f"\n[{center_name.upper()} 센터 모델 학습]")
        
        # 학습 데이터 준비
        df_raw['날짜'] = pd.to_datetime(df_raw['날짜'])
        raw_train_data = df_raw[df_raw['날짜'] <= cutoff].copy()
        
        if len(raw_train_data) < 50:
            print(f"  학습 데이터 부족")
            continue
        
        # 피처 엔지니어링
        train_data = make_features(raw_train_data, cutoff_date=cutoff_date)
        print(f"  학습 데이터: {len(train_data)}행")
        
        # 모델 평가
        training_results = comprehensive_evaluation_comparison(center_name, train_data)
        
        # 최고 성능 모델 선택 및 학습
        best_models = select_and_save_best_models(center_name, train_data, training_results, save_dir)
        
        if best_models:
            saved_models[center_name] = best_models
            
            # 학습 요약 정보 저장
            for task_type, model_info in best_models.items():
                training_summary.append({
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_info['model_name'],
                    'performance': model_info['performance'],
                    'saved_path': model_info['saved_path'],
                    'training_date': datetime.now().isoformat()
                })
    
    # 학습 요약 저장
    summary_path = os.path.join(save_dir, 'training_summary.csv')
    summary_df = pd.DataFrame(training_summary)
    summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
    
    print(f"\n={'='*60}")
    print(f"모델 학습 완료!")
    print(f"저장 위치: {save_dir}")
    print(f"학습된 모델: {len(training_summary)}개")
    print(f"학습 요약: {summary_path}")
    print(f"={'='*60}")
    
    return saved_models, training_summary

def select_and_save_best_models(center_name, train_data, training_results, save_dir):
    """센터별 최고 성능 모델 선택 및 저장"""
    results_df = pd.DataFrame(training_results)
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) == 0:
        return None
    
    saved_models = {}
    
    # 회귀 최고 성능 모델
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"  최고 회귀 모델: {best_reg['model']} (R²={best_reg['r2']:.3f})")
        
        # 모델 학습 및 저장
        model_data = train_and_save_single_model(
            center_name, train_data, best_reg, 'regression', save_dir
        )
        
        if model_data:
            saved_models['regression'] = model_data
    
    # 분류 최고 성능 모델
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"  최고 분류 모델: {best_clf['model']} (F1={best_clf['macro_f1']:.3f})")
        
        # 모델 학습 및 저장
        model_data = train_and_save_single_model(
            center_name, train_data, best_clf, 'classification', save_dir
        )
        
        if model_data:
            saved_models['classification'] = model_data
    
    return saved_models

def train_and_save_single_model(center_name, train_data, best_result, task_type, save_dir):
    """개별 모델 학습 및 저장"""
    try:
        model_name = best_result['model']
        split_method = best_result['split_method']
        target_col = "합계_1일후" if task_type == "regression" else "등급_1일후"
        
        # 전체 데이터로 학습
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=task_type, 
            test_size=0.05, split_method=split_method
        )
        
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        
        # 모델 구축 및 학습
        if task_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, task_type)
        pipeline.fit(X_all, y_all)
        
        # 모델 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{center_name}_{model_name}_{task_type}_{timestamp}.pkl"
        model_path = os.path.join(save_dir, filename)
        
        model_data = {
            'pipeline': pipeline,
            'feature_names': feature_names,
            'model_name': model_name,
            'center_name': center_name,
            'task_type': task_type,
            'performance': dict(best_result),
            'split_method': split_method,
            'training_date': datetime.now().isoformat(),
            'target_column': target_col
        }
        
        with open(model_path, 'wb') as f:
            pickle.dump(model_data, f)
        
        print(f"    저장됨: {filename}")
        
        # Feature Importance 분석 및 저장 (선택사항)
        try:
            importance_df = extract_feature_importance(pipeline, model_name, feature_names)
            if importance_df is not None:
                print(f"    Top 5 피처: {', '.join(importance_df.head(5)['feature'].tolist())}")
                
                # Feature Importance 시각화 저장
                fig = plot_feature_importance(importance_df, f"{center_name}_{model_name}")
                if fig:
                    img_path = os.path.join(save_dir, f"feature_importance_{center_name}_{model_name}_{task_type}_{timestamp}.png")
                    fig.savefig(img_path, dpi=300, bbox_inches='tight')
                    plt.close(fig)
                    print(f"    Feature Importance 저장: {os.path.basename(img_path)}")
        except Exception as e:
            print(f"    Feature Importance 분석 실패: {e}")
        
        return {
            'model_name': model_name,
            'performance': dict(best_result),
            'saved_path': model_path,
            'feature_names': feature_names,
            'target_column': target_col
        }
        
    except Exception as e:
        print(f"    모델 저장 실패: {e}")
        return None

# ================================================================================================
# 7. 2단계: 새로운 데이터로 예측 파이프라인
# ================================================================================================
def predict_with_saved_models(new_data_path, models_dir='./trained_models', 
                             historical_data_for_features=None):
    """저장된 모델들로 새로운 데이터 예측"""
    print("="*80)
    print("2단계: 새로운 데이터 예측")
    print("="*80)
    
    # 새로운 데이터 로드
    if isinstance(new_data_path, str):
        new_data = pd.read_csv(new_data_path, encoding='utf-8-sig')
        print(f"새 데이터 로드: {new_data_path} ({len(new_data)}행)")
    else:
        new_data = new_data_path.copy()
        print(f"새 데이터 제공됨: {len(new_data)}행")
    
    # 저장된 모델 목록 확인
    saved_models = load_all_saved_models(models_dir)
    
    if not saved_models:
        print("저장된 모델이 없습니다. 먼저 train_and_save_best_models()를 실행하세요.")
        return None
    
    all_predictions = []
    
    # 센터별 예측
    for center_name in saved_models.keys():
        print(f"\n[{center_name.upper()} 센터 예측]")
        
        # 해당 센터 데이터 필터링 (현재는 전체 데이터 사용)
        center_data = new_data.copy()
        
        if len(center_data) == 0:
            print(f"  해당 센터 데이터 없음")
            continue
        
        print(f"  예측 데이터: {len(center_data)}행")
        
        # 피처 생성
        if historical_data_for_features is not None:
            processed_data = make_features_for_prediction(
                historical_data_for_features[center_name], center_data
            )
        else:
            processed_data = make_simple_features(center_data)
        
        # 센터별 모델로 예측
        center_predictions = predict_for_center(center_name, processed_data, saved_models[center_name])
        all_predictions.extend(center_predictions)
    
    # 결과 정리
    if all_predictions:
        results_df = pd.DataFrame(all_predictions)
        results_df = results_df.sort_values(['center', 'date', 'task_type'])
        
        # 결과 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"predictions_{timestamp}.csv"
        results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"\n={'='*60}")
        print(f"예측 완료!")
        print(f"총 예측 건수: {len(results_df)}")
        print(f"결과 저장: {output_path}")
        print(f"={'='*60}")
        
        # 예측 결과 요약 출력
        print_prediction_summary(results_df)
        
        return results_df
    
    else:
        print("예측 결과가 없습니다.")
        return None

def load_all_saved_models(models_dir):
    """저장된 모든 모델 로드"""
    saved_models = {}
    
    if not os.path.exists(models_dir):
        return saved_models
    
    model_files = [f for f in os.listdir(models_dir) if f.endswith('.pkl') and f != 'training_summary.csv']
    
    for model_file in model_files:
        try:
            model_path = os.path.join(models_dir, model_file)
            
            with open(model_path, 'rb') as f:
                model_data = pickle.load(f)
            
            center_name = model_data['center_name']
            task_type = model_data['task_type']
            
            if center_name not in saved_models:
                saved_models[center_name] = {}
            
            saved_models[center_name][task_type] = model_data
            
            print(f"모델 로드: {center_name} {task_type} ({model_data['model_name']})")
            
        except Exception as e:
            print(f"모델 로드 실패 ({model_file}): {e}")
    
    return saved_models

def predict_for_center(center_name, processed_data, center_models):
    """센터별 모델로 예측"""
    predictions = []
    
    for task_type, model_data in center_models.items():
        try:
            pipeline = model_data['pipeline']
            feature_names = model_data['feature_names']
            model_name = model_data['model_name']
            target_col = model_data['target_column']
            
            # 예측 데이터 준비
            X_future = prepare_prediction_features(processed_data, feature_names)
            
            if X_future is None or len(X_future) == 0:
                continue
            
            # 예측 수행
            y_pred = pipeline.predict(X_future)
            
            print(f"  {task_type} 예측: {len(y_pred)}개")
            
            # 결과 저장
            for i in range(len(X_future)):
                pred_result = {
                    'date': processed_data.iloc[i]['날짜'],
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_name,
                    'target_column': target_col,
                    'predicted_value': int(y_pred[i]) if task_type == 'classification' else float(y_pred[i])
                }
                predictions.append(pred_result)
                
        except Exception as e:
            print(f"  {task_type} 예측 실패: {e}")
    
    return predictions

def print_prediction_summary(results_df):
    """예측 결과 요약 출력"""
    print(f"\n--- 예측 결과 요약 ---")
    
    for center in results_df['center'].unique():
        center_data = results_df[results_df['center'] == center]
        
        print(f"\n{center.upper()} 센터:")
        
        reg_data = center_data[center_data['task_type'] == 'regression']
        clf_data = center_data[center_data['task_type'] == 'classification']
        
        if len(reg_data) > 0:
            print(f"  회귀 예측: {len(reg_data)}개")
            print(f"  평균 예측값: {reg_data['predicted_value'].mean():.1f}")
            print(f"  예측 범위: {reg_data['predicted_value'].min():.1f} ~ {reg_data['predicted_value'].max():.1f}")
        
        if len(clf_data) > 0:
            print(f"  분류 예측: {len(clf_data)}개")
            grade_dist = clf_data['predicted_value'].value_counts().sort_index()
            print(f"  예측 등급 분포: {dict(grade_dist)}")

# ================================================================================================
# 8. 사용 가이드 및 실행 예시
# ================================================================================================
def example_usage():
    """사용 예시 가이드"""
    print("="*80)
    print("하수처리장 예측 파이프라인 사용 가이드")
    print("="*80)
    
    print("\n1단계: 모델 학습 및 저장 (한 번만 실행)")
    print("="*50)
    print("saved_models, summary = train_and_save_best_models(cutoff_date='2025-05-20')")
    print()
    print("생성되는 파일:")
    print("- ./trained_models/ 폴더")
    print("  ├── nanji_RandomForest_Reg_regression_20250101_123456.pkl")
    print("  ├── nanji_LogisticRegression_Clf_classification_20250101_123456.pkl")
    print("  ├── ... (최대 8개 모델)")
    print("  ├── training_summary.csv")
    print("  └── feature_importance_*.png (시각화 이미지들)")
    
    print("\n2단계: 새로운 데이터로 예측 (반복 실행 가능)")
    print("="*50)
    print("# CSV 파일로 예측")
    print("results = predict_with_saved_models('새로운_데이터.csv')")
    print()
    print("# DataFrame으로 예측")
    print("new_df = pd.read_csv('새로운_데이터.csv')")
    print("results = predict_with_saved_models(new_df)")
    print()
    print("# 과거 데이터와 함께 예측 (더 정확한 시차 변수)")
    print("historical_data = load_original_data()")
    print("results = predict_with_saved_models('새로운_데이터.csv',")
    print("                                   historical_data_for_features=historical_data)")
    
    print("\n새로운 데이터 형식 예시:")
    print("="*30)
    print("날짜,일_평균기온(°C),일_일강수량(mm),평균습도(%),계절,불쾌지수등급")
    print("2025-06-01,25.3,0,65,여름,약간 불쾌")
    print("2025-06-02,26.1,2.5,70,여름,불쾌")
    print("2025-06-03,24.8,15.2,75,여름,매우 불쾌")
    
    print("\n예측 결과 형식:")
    print("="*20)
    print("date,center,task_type,model_name,predicted_value")
    print("2025-06-01,nanji,regression,RandomForest_Reg,1250.5")
    print("2025-06-01,nanji,classification,LogisticRegression_Clf,2")
    
    print("\n주의사항:")
    print("="*15)
    print("- 모델 학습은 한 번만 실행하면 됩니다")
    print("- 예측은 새로운 데이터가 있을 때마다 실행 가능")
    print("- 데이터 경로가 올바른지 확인하세요")
    print("- 새 데이터의 컬럼 구조는 기존 학습 데이터와 유사해야 합니다")

if __name__ == "__main__":
    example_usage()

# ================================================================================================
# 실행 준비 완료
# ================================================================================================
print("\n" + "="*80)
print("완전한 하수처리장 예측 파이프라인 준비 완료!")
print("="*80)
print("\n다음 명령어로 시작하세요:")
print("\n# 1단계: 모델 학습")
print("saved_models, summary = train_and_save_best_models()")
print("\n# 2단계: 예측")
print("results = predict_with_saved_models('새로운_데이터.csv')")
print("\n사용 가이드: example_usage()")
print("="*80)

하수처리장 예측 파이프라인 초기화 완료
XGBoost: ✓
LightGBM: ✓
CatBoost: ✓
SHAP: ✓
하수처리장 예측 파이프라인 사용 가이드

1단계: 모델 학습 및 저장 (한 번만 실행)
saved_models, summary = train_and_save_best_models(cutoff_date='2025-05-20')

생성되는 파일:
- ./trained_models/ 폴더
  ├── nanji_RandomForest_Reg_regression_20250101_123456.pkl
  ├── nanji_LogisticRegression_Clf_classification_20250101_123456.pkl
  ├── ... (최대 8개 모델)
  ├── training_summary.csv
  └── feature_importance_*.png (시각화 이미지들)

2단계: 새로운 데이터로 예측 (반복 실행 가능)
# CSV 파일로 예측
results = predict_with_saved_models('새로운_데이터.csv')

# DataFrame으로 예측
new_df = pd.read_csv('새로운_데이터.csv')
results = predict_with_saved_models(new_df)

# 과거 데이터와 함께 예측 (더 정확한 시차 변수)
historical_data = load_original_data()
results = predict_with_saved_models('새로운_데이터.csv',
                                   historical_data_for_features=historical_data)

새로운 데이터 형식 예시:
날짜,일_평균기온(°C),일_일강수량(mm),평균습도(%),계절,불쾌지수등급
2025-06-01,25.3,0,65,여름,약간 불쾌
2025-06-02,26.1,2.5,70,여름,불쾌
2025-06-03,24.8,15.2,75,여름,매우 불쾌

예측 결과 형식:
d

In [2]:
# 1. 위 코드 전체 실행 (함수들 로드)

# 2. 모델 학습 (한 번만)
saved_models, summary = train_and_save_best_models()

# 3. 예측 (필요할 때마다)
# results = predict_with_saved_models('새_데이터.csv')

1단계: 모델 학습 및 저장

[NANJI 센터 모델 학습]
  학습 데이터: 3058행

센터: nanji - Stratified vs 시계열 분할 비교
데이터 크기: 3058행, 44컬럼
등급 분포: {0.0: 458, 1.0: 1682, 2.0: 611, 3.0: 307}
클래스 불균형 비율: 5.5:1 (최대:1682, 최소:307)

분할 방법: TEMPORAL

--- 회귀 모델 평가 (temporal) ---
회귀용 데이터: 학습 2446행, 테스트 612행


회귀(temporal):  17%|█▋        | 1/6 [00:00<00:04,  1.24it/s]

  RandomForest_Reg  : R²=0.557, MAE=49542, MAPE=7.3%
  LinearRegression  : R²=0.513, MAE=62439, MAPE=9.7%


회귀(temporal):  50%|█████     | 3/6 [00:02<00:02,  1.22it/s]

  GradientBoosting_Reg: R²=0.395, MAE=58665, MAPE=9.0%


...

  XGBoost_Clf       : ACC=0.658, F1=0.547, 극값F1=0.488


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
분류(stratified):  83%|████████▎ | 5/6 [00:15<00:03,  3.51s/it]

  LightGBM_Clf      : ACC=0.647, F1=0.545, 극값F1=0.515


  CatBoost_Clf      : ACC=0.614, F1=0.559, 극값F1=0.577
  최고 회귀 모델: CatBoost_Reg (R²=0.515)


    저장됨: tancheon_CatBoost_Reg_regression_20250827_123729.pkl
    Top 5 피처: 세탁업, 체력단련장업, 강수량_7일_누적, 하천, 일_일강수량(mm)
    Feature Importance 저장: feature_importance_tancheon_CatBoost_Reg_regression_20250827_123729.png
  최고 분류 모델: CatBoost_Clf (F1=0.559)
    저장됨: tancheon_CatBoost_Clf_classification_20250827_123730.pkl
    Top 5 피처: 세탁업, 강수량_7일_누적, 생활인구, 월, 체력단련장업
    Feature Importance 저장: feature_importance_tancheon_CatBoost_Clf_classification_20250827_123730.png

모델 학습 완료!
저장 위치: ./trained_models
학습된 모델: 8개
학습 요약: ./trained_models/training_summary.csv


# 다시 버전 13

**1단계 결과 (모델 학습):**

**파일:**
- `./trained_models/nanji_RandomForest_Reg_regression_20250101_123456.pkl` (난지 회귀 모델)
- `./trained_models/nanji_LogisticRegression_Clf_classification_20250101_123456.pkl` (난지 분류 모델)
- `./trained_models/jungnang_XGBoost_Reg_regression_20250101_123456.pkl` (중랑 회귀 모델)
- `./trained_models/jungnang_RandomForest_Clf_classification_20250101_123456.pkl` (중랑 분류 모델)
- ... (서남, 탄천도 각각 2개씩 = 총 8개 모델)
- `./trained_models/training_summary.csv` (학습 결과 요약)
- `./results/feature_importance_nanji_RandomForest_Reg_regression_*.png` (총 8개 시각화)

**콘솔 출력:**
```
난지 센터 최고 회귀 모델: RandomForest_Reg (R²=0.851)
난지 센터 최고 분류 모델: LogisticRegression_Clf (F1=0.743)
중랑 센터 최고 회귀 모델: XGBoost_Reg (R²=0.823)
...
```

**2단계 결과 (예측):**

**파일:**
- `./results/predictions_20250101_143022.csv`

**predictions CSV 내용:**
```csv
date,center,task_type,model_name,target_column,predicted_value
2025-06-01,nanji,regression,RandomForest_Reg,합계_1일후,1247.3
2025-06-01,nanji,classification,LogisticRegression_Clf,등급_1일후,2
2025-06-01,jungnang,regression,XGBoost_Reg,합계_1일후,982.1
2025-06-01,jungnang,classification,RandomForest_Clf,등급_1일후,1
2025-06-01,seonam,regression,LightGBM_Reg,합계_1일후,1456.8
2025-06-01,seonam,classification,XGBoost_Clf,등급_1일후,3
2025-06-01,tancheon,regression,CatBoost_Reg,합계_1일후,678.9
2025-06-01,tancheon,classification,GradientBoosting_Clf,등급_1일후,1
```

**콘솔 출력:**
```
NANJI 센터:
  회귀 예측: 10개
  평균 예측값: 1234.5
  예측 범위: 980.2 ~ 1456.7
  분류 예측: 10개
  예측 등급 분포: {0: 2, 1: 4, 2: 3, 3: 1}
```

**핵심:**
- **predicted_value**: 실제 예측값 (회귀는 하수처리량, 분류는 0~3 등급)
- **4개 센터 × 2개 태스크 × N일** 만큼 예측 결과 생성

In [7]:
# ================================================================================================
# 완전한 하수처리장 예측 파이프라인 (학습-저장-예측)
# ================================================================================================

# 필수 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score
from tqdm import tqdm
import time, os, json, warnings
import joblib
import pickle
from datetime import datetime

# 선택적 라이브러리 import
try:
    import xgboost as xgb
    HAS_XGB = True
except ImportError:
    HAS_XGB = False

try:
    import lightgbm as lgb
    HAS_LGB = True
except ImportError:
    HAS_LGB = False

try:
    import catboost as cb
    HAS_CATBOOST = True
except ImportError:
    HAS_CATBOOST = False

try:
    import shap
    HAS_SHAP = True
except ImportError:
    HAS_SHAP = False

# 한글 폰트 설정
warnings.filterwarnings("ignore")
try:
    plt.rcParams['font.family'] = 'AppleGothic' # 맥
except Exception:
    plt.rcParams['font.family'] ='Malgun Gothic' # 윈도우
plt.rcParams['axes.unicode_minus'] = False

print("="*80)
print("하수처리장 예측 파이프라인 초기화 완료")
print(f"XGBoost: {'✓' if HAS_XGB else '✗'}")
print(f"LightGBM: {'✓' if HAS_LGB else '✗'}")
print(f"CatBoost: {'✓' if HAS_CATBOOST else '✗'}")
print(f"SHAP: {'✓' if HAS_SHAP else '✗'}")
print("="*80)

# ================================================================================================
# 1. 모델 정의 함수들
# ================================================================================================
def build_regression_models():
    """회귀 모델들"""
    models = {}
    models["RandomForest_Reg"] = RandomForestRegressor(
        n_estimators=300, min_samples_leaf=2, random_state=42, n_jobs=-1
    )
    models["LinearRegression"] = LinearRegression()
    models["GradientBoosting_Reg"] = GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    if HAS_XGB:
        models["XGBoost_Reg"] = xgb.XGBRegressor(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=0
        )
    if HAS_LGB:
        models["LightGBM_Reg"] = lgb.LGBMRegressor(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            random_state=42, n_jobs=-1, verbosity=-1
        )
    if HAS_CATBOOST:
        models["CatBoost_Reg"] = cb.CatBoostRegressor(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False
        )
    return models

def build_classification_models():
    """분류 모델들 (4등급)"""
    models = {}
    models["RandomForest_Clf"] = RandomForestClassifier(
        n_estimators=300, min_samples_leaf=2, random_state=42, 
        n_jobs=-1, class_weight='balanced'
    ) 
    models["GradientBoosting_Clf"] = GradientBoostingClassifier(
        n_estimators=200, learning_rate=0.1, random_state=42
    )
    models["LogisticRegression_Clf"] = LogisticRegression(
        multi_class="multinomial", solver="lbfgs", max_iter=1000,
        random_state=42, class_weight='balanced'
    )
    if HAS_XGB:
        models["XGBoost_Clf"] = xgb.XGBClassifier(
            n_estimators=400, max_depth=5, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multi:softprob", num_class=4,
            tree_method="hist", random_state=42, n_jobs=-1, verbosity=0
        )
    if HAS_LGB:
        models["LightGBM_Clf"] = lgb.LGBMClassifier(
            n_estimators=500, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8,
            objective="multiclass", num_class=4,
            random_state=42, n_jobs=-1, verbosity=-1, is_unbalance=True
        )
    if HAS_CATBOOST:
        models["CatBoost_Clf"] = cb.CatBoostClassifier(
            iterations=500, learning_rate=0.05, depth=6,
            random_state=42, verbose=False, auto_class_weights='Balanced'
        )
    return models

# ================================================================================================
# 2. 데이터 처리 함수들
# ================================================================================================
def make_pipeline_unified(model, model_name, model_type):
    """통합 전처리 파이프라인"""
    if model_name in ["LinearRegression", "LogisticRegression_Clf"]:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
        ])
    else:
        pre = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
        ])
    return Pipeline(steps=[("pre", pre), ("model", model)])

def prepare_data_stratified(df, target_col, model_type, test_size=0.2, split_method='stratified'):
    """데이터 준비 - Stratified vs 시계열 분할"""
    work = df.sort_values('날짜').reset_index(drop=True).copy()
    dates = pd.to_datetime(work['날짜'])

    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in work.columns]
    X_raw = work.drop(columns=drop_cols, errors="ignore")
    
    for c in X_raw.columns:
        X_raw[c] = pd.to_numeric(X_raw[c], errors="coerce")

    if model_type == "regression":
        y = pd.to_numeric(work[target_col], errors="coerce")
    else:
        y = work[target_col].astype("int64")

    valid_idx = (~X_raw.isnull().all(axis=1)) & (~pd.isnull(y))
    X_raw = X_raw[valid_idx].reset_index(drop=True)
    y = y[valid_idx].reset_index(drop=True)
    dates = dates[valid_idx].reset_index(drop=True)
    
    if split_method == 'stratified':
        if model_type == "classification":
            sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)
            train_idx, test_idx = next(sss.split(X_raw, y))
        else:
            train_idx, test_idx = train_test_split(
                range(len(X_raw)), test_size=test_size, random_state=42
            )
            
        X_train, X_test = X_raw.iloc[train_idx].copy(), X_raw.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
        dates_train, dates_test = dates.iloc[train_idx].copy(), dates.iloc[test_idx].copy()
        
    else:  # temporal split
        n = len(X_raw)
        split = int(n * (1 - test_size))
        X_train, X_test = X_raw.iloc[:split].copy(), X_raw.iloc[split:].copy()
        y_train, y_test = y.iloc[:split].copy(), y.iloc[split:].copy()
        dates_train, dates_test = dates.iloc[:split].copy(), dates.iloc[split:].copy()

    feature_names = list(X_raw.columns)
    return X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test

def make_features(df, cutoff_date=None):
    """파생변수 생성 함수 - Data Leakage 방지 버전"""
    df = df.copy()
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)

    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday

    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    df['계절'] = df['계절'].map(season_map).astype('Int64')
    df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')

    # 강수량 시차 피처
    df['강수량_1일전'] = df['일_일강수량(mm)'].shift(1)
    df['강수량_2일전'] = df['일_일강수량(mm)'].shift(2)
    df['강수량_1일_누적'] = df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    df['강수량_2일_누적'] = df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    df['강수량_3일_누적'] = df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    df['강수량_5일_누적'] = df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    df['강수량_7일_누적'] = df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()

    df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산 (간단 버전)
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 분류용 등급 계산
    q = df['합계'].dropna().quantile([0.15, 0.70, 0.90])
    q15, q70, q90 = float(q.loc[0.15]), float(q.loc[0.70]), float(q.loc[0.90])

    def categorize(x):
        if pd.isna(x):
            return np.nan
        if x < q15:
            return 0
        elif x < q70:
            return 1
        elif x < q90:
            return 2
        else:
            return 3

    df['등급'] = df['합계'].apply(categorize)
    
    # 타겟 변수 생성 (Data Leakage 방지)
    if cutoff_date is not None:
        cutoff = pd.to_datetime(cutoff_date)
        
        df['합계_1일후'] = np.nan
        df['합계_2일후'] = np.nan
        df['등급_1일후'] = np.nan
        df['등급_2일후'] = np.nan
        
        for i in range(len(df)):
            current_date = df.loc[i, '날짜']
            
            if i + 1 < len(df) and current_date <= cutoff:
                next_date = df.loc[i+1, '날짜']
                if next_date <= cutoff:
                    df.loc[i, '합계_1일후'] = df.loc[i+1, '합계']
                    df.loc[i, '등급_1일후'] = df.loc[i+1, '등급']
            
            if i + 2 < len(df) and current_date <= cutoff:
                next2_date = df.loc[i+2, '날짜']
                if next2_date <= cutoff:
                    df.loc[i, '합계_2일후'] = df.loc[i+2, '합계']
                    df.loc[i, '등급_2일후'] = df.loc[i+2, '등급']
    else:
        df['합계_1일후'] = df['합계'].shift(-1)
        df['합계_2일후'] = df['합계'].shift(-2)
        df['등급_1일후'] = df['등급'].shift(-1).astype('Int64')
        df['등급_2일후'] = df['등급'].shift(-2).astype('Int64')

    df.attrs['cutoffs'] = {"q15": q15, "q70": q70, "q90": q90}
    df = df.dropna().reset_index(drop=True)
    df = df[df["날짜"] < "2025-06-01"]
    
    return df

def make_features_for_prediction(historical_df, future_df):
    """새로운 데이터에 대한 파생변수 생성 (과거 데이터 활용)"""
    combined_df = pd.concat([historical_df, future_df], ignore_index=True)
    combined_df['날짜'] = pd.to_datetime(combined_df['날짜'])
    combined_df = combined_df.sort_values('날짜').reset_index(drop=True)
    
    combined_df['월'] = combined_df['날짜'].dt.month
    combined_df['요일'] = combined_df['날짜'].dt.weekday
    
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    combined_df['계절'] = combined_df['계절'].map(season_map).astype('Int64')
    combined_df['불쾌지수등급'] = combined_df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 시차 변수들
    combined_df['강수량_1일전'] = combined_df['일_일강수량(mm)'].shift(1)
    combined_df['강수량_2일전'] = combined_df['일_일강수량(mm)'].shift(2)
    combined_df['강수량_1일_누적'] = combined_df['일_일강수량(mm)'].rolling(1, min_periods=1).sum()
    combined_df['강수량_2일_누적'] = combined_df['일_일강수량(mm)'].rolling(2, min_periods=1).sum()
    combined_df['강수량_3일_누적'] = combined_df['일_일강수량(mm)'].rolling(3, min_periods=1).sum()
    combined_df['강수량_5일_누적'] = combined_df['일_일강수량(mm)'].rolling(5, min_periods=1).sum()
    combined_df['강수량_7일_누적'] = combined_df['일_일강수량(mm)'].rolling(7, min_periods=1).sum()
    
    combined_df['일교차'] = combined_df['일_최고기온(°C)'] - combined_df['일_최저기온(°C)']
    combined_df['폭우_여부'] = (combined_df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(combined_df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(combined_df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(combined_df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    combined_df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    # 새 데이터 부분만 반환
    historical_len = len(historical_df)
    return combined_df.iloc[historical_len:].reset_index(drop=True)

def make_simple_features(data):
    """간단한 피처 생성 (시차 변수 제외)"""
    df = data.copy()
    df['날짜'] = pd.to_datetime(df['날짜'])
    df = df.sort_values('날짜').reset_index(drop=True)
    
    # 기본 피처들
    df['월'] = df['날짜'].dt.month
    df['요일'] = df['날짜'].dt.weekday
    
    # 계절/불쾌지수 매핑
    season_map = {'봄': 0, '여름': 1, '가을': 2, '겨울': 3}
    discomfort_map = {'쾌적': 0, '약간 불쾌': 1, '불쾌': 2, '매우 불쾌': 3, '극심한 불쾌': 4}
    
    if '계절' in df.columns:
        df['계절'] = df['계절'].map(season_map).astype('Int64')
    if '불쾌지수등급' in df.columns:
        df['불쾌지수등급'] = df['불쾌지수등급'].map(discomfort_map).astype('Int64')
    
    # 기본 계산 피처들
    if '일_최고기온(°C)' in df.columns and '일_최저기온(°C)' in df.columns:
        df['일교차'] = df['일_최고기온(°C)'] - df['일_최저기온(°C)']
    
    if '일_일강수량(mm)' in df.columns:
        df['폭우_여부'] = (df['일_일강수량(mm)'] >= 80).astype(int)
    
    # 체감온도 계산
    T = pd.to_numeric(df.get('일_평균기온(°C)', np.nan), errors='coerce')
    V_ms = pd.to_numeric(df.get('일_평균풍속(m/s)', np.nan), errors='coerce')
    RH = pd.to_numeric(df.get('평균습도(%)', np.nan), errors='coerce')
    
    e = (RH/100.0) * 6.105 * np.exp(17.27*T/(237.7 + T))
    df['체감온도(°C)'] = T + 0.33*e - 0.70*V_ms - 4.00
    
    return df

# ================================================================================================
# 3. 평가 함수들
# ================================================================================================
def evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test):
    """회귀 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "regression")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100
        
        return {
            'model': model_name,
            'type': 'regression',
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'mape': mape,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'regression',
            'mae': np.nan,
            'rmse': np.nan,
            'r2': np.nan,
            'mape': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def evaluate_classification_model(model, model_name, X_train, X_test, y_train, y_test):
    """분류 모델 평가"""
    try:
        pipe = make_pipeline_unified(model, model_name, "classification")
        pipe.fit(X_train, y_train)
        
        y_pred = pipe.predict(X_test)
        
        if isinstance(y_pred, np.ndarray) and y_pred.ndim > 1:
            y_pred = y_pred.ravel()
        
        acc = accuracy_score(y_test, y_pred)
        f1_macro = f1_score(y_test, y_pred, average="macro", zero_division=0)
        f1_weighted = f1_score(y_test, y_pred, average="weighted", zero_division=0)
        
        extreme_classes = [0, 3]
        y_true_extreme = pd.Series(y_test).isin(extreme_classes).astype(int)
        y_pred_extreme = pd.Series(y_pred).isin(extreme_classes).astype(int)
        extreme_f1 = f1_score(y_true_extreme, y_pred_extreme, zero_division=0)
        
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': acc,
            'macro_f1': f1_macro,
            'weighted_f1': f1_weighted,
            'extreme_f1': extreme_f1,
            'success': True
        }, pipe, y_pred
        
    except Exception as e:
        return {
            'model': model_name,
            'type': 'classification',
            'accuracy': np.nan,
            'macro_f1': np.nan,
            'weighted_f1': np.nan,
            'extreme_f1': np.nan,
            'success': False,
            'error': str(e)
        }, None, None

def comprehensive_evaluation_comparison(center_name, df):
    """Stratified vs 시계열 분할 비교 평가"""
    print(f"\n{'='*70}")
    print(f"센터: {center_name} - Stratified vs 시계열 분할 비교")
    print(f"{'='*70}")
    
    print(f"데이터 크기: {len(df)}행, {len(df.columns)}컬럼")
    
    if '등급_1일후' in df.columns:
        grade_dist = df['등급_1일후'].value_counts().sort_index()
        print(f"등급 분포: {dict(grade_dist)}")
        
        min_class = grade_dist.min()
        max_class = grade_dist.max()
        imbalance_ratio = max_class / min_class
        print(f"클래스 불균형 비율: {imbalance_ratio:.1f}:1 (최대:{max_class}, 최소:{min_class})")
    
    results = []
    
    for split_method in ['temporal', 'stratified']:
        print(f"\n{'='*50}")
        print(f"분할 방법: {split_method.upper()}")
        print(f"{'='*50}")
        
        # 회귀 모델 평가
        reg_method_name = "random_shuffle" if split_method == "stratified" else split_method
        print(f"\n--- 회귀 모델 평가 ({reg_method_name}) ---")
        
        try:
            X_train, X_test, y_train, y_test, feature_names, dates_train, dates_test = prepare_data_stratified(
                df, target_col="합계_1일후", model_type="regression", test_size=0.2, split_method=split_method
            )
            
            print(f"회귀용 데이터: 학습 {len(X_train)}행, 테스트 {len(X_test)}행")
            
            regression_models = build_regression_models()
            
            for model_name, model in tqdm(regression_models.items(), desc=f"회귀({reg_method_name})", leave=False):
                result, pipe, y_pred = evaluate_regression_model(model, model_name, X_train, X_test, y_train, y_test)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: R²={result['r2']:.3f}, MAE={result['mae']:.0f}, MAPE={result['mape']:.1f}%")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"회귀 모델 평가 실패 ({reg_method_name}): {e}")
        
        # 분류 모델 평가
        print(f"\n--- 분류 모델 평가 ({split_method}) ---")
        
        try:
            X_train_clf, X_test_clf, y_train_clf, y_test_clf, feature_names_clf, _, _ = prepare_data_stratified(
                df, target_col="등급_1일후", model_type="classification", test_size=0.2, split_method=split_method
            )
            
            print(f"분류용 데이터: 학습 {len(X_train_clf)}행, 테스트 {len(X_test_clf)}행")
            
            test_dist = pd.Series(y_test_clf).value_counts().sort_index()
            train_dist = pd.Series(y_train_clf).value_counts().sort_index()
            print(f"학습 세트 등급 분포: {dict(train_dist)}")
            print(f"테스트 세트 등급 분포: {dict(test_dist)}")
            
            classification_models = build_classification_models()
            
            for model_name, model in tqdm(classification_models.items(), desc=f"분류({split_method})", leave=False):
                result, pipe, y_pred = evaluate_classification_model(model, model_name, X_train_clf, X_test_clf, y_train_clf, y_test_clf)
                result['center'] = center_name
                result['split_method'] = split_method
                results.append(result)
                
                if result['success']:
                    print(f"  {model_name:18s}: ACC={result['accuracy']:.3f}, F1={result['macro_f1']:.3f}, 극값F1={result['extreme_f1']:.3f}")
                else:
                    print(f"  {model_name:18s}: 실패 - {result.get('error', '')[:50]}")
                    
        except Exception as e:
            print(f"분류 모델 평가 실패 ({split_method}): {e}")
    
    return results

# ================================================================================================
# 4. Feature Importance & SHAP 분석 함수들 (선택사항)
# ================================================================================================
def extract_feature_importance(model, model_name, feature_names):
    """모델별 Feature Importance 추출"""
    try:
        mdl = model.named_steps['model']
        if hasattr(mdl, 'feature_importances_'):
            importance = mdl.feature_importances_
        elif hasattr(mdl, 'coef_'):
            coef = mdl.coef_
            if isinstance(coef, np.ndarray) and coef.ndim == 2:
                importance = np.mean(np.abs(coef), axis=0)
            else:
                importance = np.abs(coef)
        else:
            return None

        if len(importance) != len(feature_names):
            print(f"[경고] importance 길이({len(importance)}) != feature_names({len(feature_names)})")
            m = min(len(importance), len(feature_names))
            importance = np.asarray(importance)[:m]
            feature_names = list(feature_names)[:m]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)

        return importance_df
    except Exception as e:
        print(f"Feature importance 추출 실패 ({model_name}): {e}")
        return None

def plot_feature_importance(importance_df, model_name, top_n=15):
    """Feature Importance 시각화"""
    if importance_df is None or len(importance_df) == 0:
        return None
    
    fig, ax = plt.subplots(figsize=(10, 8))
    top_features = importance_df.head(top_n)
    
    ax.barh(range(len(top_features)), top_features['importance'], color='skyblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.set_xlabel('Importance')
    ax.set_title(f'{model_name} - Top {top_n} Feature Importance')
    ax.invert_yaxis()
    
    for i, v in enumerate(top_features['importance']):
        ax.text(v + 0.001, i, f'{v:.3f}', va='center')
    
    fig.tight_layout()
    return fig

def analyze_model_with_shap(model, X_test, feature_names, model_name, max_samples=100):
    """SHAP 분석"""
    if not HAS_SHAP:
        print("SHAP 라이브러리가 설치되지 않았습니다.")
        return None
    
    try:
        if len(X_test) > max_samples:
            sample_idx = np.random.choice(len(X_test), max_samples, replace=False)
            X_sample = X_test.iloc[sample_idx]
        else:
            X_sample = X_test
        
        X_processed = model.named_steps['pre'].transform(X_sample)
        
        if 'RandomForest' in model_name or 'GradientBoosting' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'XGBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'LightGBM' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        elif 'CatBoost' in model_name:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        else:
            explainer = shap.LinearExplainer(model.named_steps['model'], X_processed)
        
        shap_values = explainer.shap_values(X_processed)
        
        if isinstance(shap_values, list):
            shap_values = shap_values[0]
        
        return shap_values, X_processed, explainer
        
    except Exception as e:
        print(f"SHAP 분석 실패 ({model_name}): {e}")
        return None

def plot_shap_summary(shap_values, X_processed, feature_names, model_name):
    """SHAP Summary Plot"""
    if shap_values is None or not HAS_SHAP:
        return []

    figs = []
    try:
        # Bar plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_processed,
                          feature_names=feature_names,
                          plot_type="bar", show=False)
        ax = plt.gca()
        ax.set_title(f'{model_name} - SHAP Feature Importance')
        figs.append(plt.gcf())

        # Beeswarm plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_processed,
                          feature_names=feature_names,
                          show=False)
        ax = plt.gca()
        ax.set_title(f'{model_name} - SHAP Summary Plot')
        figs.append(plt.gcf())

    except Exception as e:
        print(f"SHAP 시각화 실패 ({model_name}): {e}")

    return figs

# ================================================================================================
# 5. 유틸리티 함수들
# ================================================================================================
def load_original_data():
    """원본 데이터 로드"""
    nanji_raw = pd.read_csv('../data/processed/center_season/nanji/난지_merged.csv', encoding='utf-8-sig')
    jungnang_raw = pd.read_csv('../data/processed/center_season/jungnang/중랑_merged.csv', encoding='utf-8-sig')
    seonam_raw = pd.read_csv('../data/processed/center_season/seonam/서남_merged.csv', encoding='utf-8-sig')
    tancheon_raw = pd.read_csv('../data/processed/center_season/tancheon/탄천_merged.csv', encoding='utf-8-sig')
    
    return {
        "nanji": nanji_raw,
        "jungnang": jungnang_raw,
        "seonam": seonam_raw,
        "tancheon": tancheon_raw
    }

def prepare_prediction_features(future_data, expected_features):
    """예측용 피처 준비"""
    not_use_col = [
        '날짜', '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    available_cols = [col for col in future_data.columns if col not in not_use_col]
    X_future = future_data[available_cols].copy()
    
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    X_future = X_future[expected_features].copy()
    return X_future

# ================================================================================================
# 6. 1단계: 모델 학습 및 저장 파이프라인
# ================================================================================================
def train_and_save_best_models(cutoff_date='2025-05-20', save_dir='./trained_models', results_dir='./results'):
    """각 센터별 최고 성능 모델 학습 및 저장"""
    print("="*80)
    print("1단계: 모델 학습 및 저장")
    print("="*80)
    
    # 저장 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    
    # 데이터 로드
    centers_data = load_original_data()
    cutoff = pd.to_datetime(cutoff_date)
    
    saved_models = {}
    training_summary = []
    
    for center_name, df_raw in centers_data.items():
        print(f"\n[{center_name.upper()} 센터 모델 학습]")
        
        # 학습 데이터 준비
        df_raw['날짜'] = pd.to_datetime(df_raw['날짜'])
        raw_train_data = df_raw[df_raw['날짜'] <= cutoff].copy()
        
        if len(raw_train_data) < 50:
            print(f"  학습 데이터 부족")
            continue
        
        # 피처 엔지니어링
        train_data = make_features(raw_train_data, cutoff_date=cutoff_date)
        print(f"  학습 데이터: {len(train_data)}행")
        
        # 모델 평가
        training_results = comprehensive_evaluation_comparison(center_name, train_data)
        
        # 최고 성능 모델 선택 및 학습
        best_models = select_and_save_best_models(center_name, train_data, training_results, save_dir, results_dir)
        
        if best_models:
            saved_models[center_name] = best_models
            
            # 학습 요약 정보 저장
            for task_type, model_info in best_models.items():
                training_summary.append({
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_info['model_name'],
                    'performance': model_info['performance'],
                    'saved_path': model_info['saved_path'],
                    'training_date': datetime.now().isoformat()
                })
    
    # 학습 요약 저장
    summary_path = os.path.join(save_dir, 'training_summary.csv')
    summary_df = pd.DataFrame(training_summary)
    summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
    
    print(f"\n={'='*60}")
    print(f"모델 학습 완료!")
    print(f"저장 위치: {save_dir}")
    print(f"학습된 모델: {len(training_summary)}개")
    print(f"학습 요약: {summary_path}")
    print(f"={'='*60}")
    
    return saved_models, training_summary

def select_and_save_best_models(center_name, train_data, training_results, save_dir, results_dir):
    """센터별 최고 성능 모델 선택 및 저장"""
    results_df = pd.DataFrame(training_results)
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) == 0:
        return None
    
    saved_models = {}
    
    # 회귀 최고 성능 모델
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"  최고 회귀 모델: {best_reg['model']} (R²={best_reg['r2']:.3f})")
        
        # 모델 학습 및 저장
        model_data = train_and_save_single_model(
            center_name, train_data, best_reg, 'regression', save_dir, results_dir
        )
        
        if model_data:
            saved_models['regression'] = model_data
    
    # 분류 최고 성능 모델
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"  최고 분류 모델: {best_clf['model']} (F1={best_clf['macro_f1']:.3f})")
        
        # 모델 학습 및 저장
        model_data = train_and_save_single_model(
            center_name, train_data, best_clf, 'classification', save_dir, results_dir
        )
        
        if model_data:
            saved_models['classification'] = model_data
    
    return saved_models

def train_and_save_single_model(center_name, train_data, best_result, task_type, save_dir, results_dir):
    """개별 모델 학습 및 저장"""
    try:
        model_name = best_result['model']
        split_method = best_result['split_method']
        target_col = "합계_1일후" if task_type == "regression" else "등급_1일후"
        
        # 전체 데이터로 학습
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=task_type, 
            test_size=0.05, split_method=split_method
        )
        
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        
        # 모델 구축 및 학습
        if task_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, task_type)
        pipeline.fit(X_all, y_all)
        
        # 모델 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{center_name}_{model_name}_{task_type}_{timestamp}.pkl"
        model_path = os.path.join(save_dir, filename)
        
        model_data = {
            'pipeline': pipeline,
            'feature_names': feature_names,
            'model_name': model_name,
            'center_name': center_name,
            'task_type': task_type,
            'performance': dict(best_result),
            'split_method': split_method,
            'training_date': datetime.now().isoformat(),
            'target_column': target_col
        }
        
        with open(model_path, 'wb') as f:
            pickle.dump(model_data, f)
        
        print(f"    저장됨: {filename}")
        
        # Feature Importance 분석 및 저장 (선택사항)
        try:
            importance_df = extract_feature_importance(pipeline, model_name, feature_names)
            if importance_df is not None:
                print(f"    Top 5 피처: {', '.join(importance_df.head(5)['feature'].tolist())}")
                
                # Feature Importance 시각화 저장
                fig = plot_feature_importance(importance_df, f"{center_name}_{model_name}")
                if fig:
                    img_path = os.path.join(results_dir, f"feature_importance_{center_name}_{model_name}_{task_type}_{timestamp}.png")
                    fig.savefig(img_path, dpi=300, bbox_inches='tight')
                    plt.close(fig)
                    print(f"    Feature Importance 저장: {os.path.basename(img_path)}")
        except Exception as e:
            print(f"    Feature Importance 분석 실패: {e}")
        
        return {
            'model_name': model_name,
            'performance': dict(best_result),
            'saved_path': model_path,
            'feature_names': feature_names,
            'target_column': target_col
        }
        
    except Exception as e:
        print(f"    모델 저장 실패: {e}")
        return None
    
# ================================================================================================
# 모델 성능 비교 시각화 추가 코드 (버전 13에 추가)
# ================================================================================================

def create_performance_comparison_plots(training_summary, results_dir='./results'):
    """모델 성능 비교 시각화 생성"""
    
    print("\n모델 성능 비교 그래프 생성 중...")
    
    if not training_summary or len(training_summary) == 0:
        print("비교할 성능 데이터가 없습니다.")
        return
    
    # DataFrame 변환
    if isinstance(training_summary, list):
        summary_df = pd.DataFrame(training_summary)
    else:
        summary_df = training_summary.copy()
    
    # 결과 저장 디렉토리 확인
    os.makedirs(results_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 센터별 회귀 성능 비교
    plot_regression_performance_comparison(summary_df, results_dir, timestamp)
    
    # 2. 센터별 분류 성능 비교  
    plot_classification_performance_comparison(summary_df, results_dir, timestamp)
    
    # 3. 모델별 성능 비교 (전체 센터 통합)
    plot_model_performance_comparison(summary_df, results_dir, timestamp)
    
    # 4. 센터별 최고 성능 요약
    plot_best_performance_summary(summary_df, results_dir, timestamp)
    
    print(f"성능 비교 그래프 저장 완료: {results_dir}")

def plot_regression_performance_comparison(summary_df, results_dir, timestamp):
    """회귀 모델 성능 비교 (R², MAE)"""
    
    reg_data = summary_df[summary_df['task_type'] == 'regression'].copy()
    if len(reg_data) == 0:
        return
    
    # 성능 지표 추출
    reg_data['r2'] = reg_data['performance'].apply(lambda x: x.get('r2', 0))
    reg_data['mae'] = reg_data['performance'].apply(lambda x: x.get('mae', 0))
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('회귀 모델 성능 비교', fontsize=16, fontweight='bold')
    
    # R² 비교 (센터별)
    pivot_r2 = reg_data.pivot_table(index='center', columns='model_name', values='r2', fill_value=0)
    pivot_r2.plot(kind='bar', ax=axes[0,0], color=plt.cm.Set3.colors)
    axes[0,0].set_title('센터별 R² 성능')
    axes[0,0].set_ylabel('R² Score')
    axes[0,0].set_ylim(0, 1)
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # MAE 비교 (센터별)  
    pivot_mae = reg_data.pivot_table(index='center', columns='model_name', values='mae', fill_value=0)
    pivot_mae.plot(kind='bar', ax=axes[0,1], color=plt.cm.Set2.colors)
    axes[0,1].set_title('센터별 MAE 성능 (낮을수록 좋음)')
    axes[0,1].set_ylabel('MAE')
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 모델별 평균 R²
    model_r2_avg = reg_data.groupby('model_name')['r2'].mean().sort_values(ascending=False)
    model_r2_avg.plot(kind='bar', ax=axes[1,0], color='skyblue')
    axes[1,0].set_title('모델별 평균 R² 성능')
    axes[1,0].set_ylabel('평균 R² Score')
    axes[1,0].set_ylim(0, 1)
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # 값 표시
    for i, v in enumerate(model_r2_avg.values):
        axes[1,0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # 모델별 평균 MAE
    model_mae_avg = reg_data.groupby('model_name')['mae'].mean().sort_values(ascending=True)
    model_mae_avg.plot(kind='bar', ax=axes[1,1], color='lightcoral')
    axes[1,1].set_title('모델별 평균 MAE 성능')
    axes[1,1].set_ylabel('평균 MAE')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    # 값 표시
    for i, v in enumerate(model_mae_avg.values):
        axes[1,1].text(i, v + max(model_mae_avg)*0.01, f'{v:.1f}', ha='center', va='bottom')
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"regression_performance_comparison_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  회귀 성능 비교 저장: {os.path.basename(save_path)}")

def plot_classification_performance_comparison(summary_df, results_dir, timestamp):
    """분류 모델 성능 비교 (Accuracy, F1)"""
    
    clf_data = summary_df[summary_df['task_type'] == 'classification'].copy()
    if len(clf_data) == 0:
        return
    
    # 성능 지표 추출
    clf_data['accuracy'] = clf_data['performance'].apply(lambda x: x.get('accuracy', 0))
    clf_data['macro_f1'] = clf_data['performance'].apply(lambda x: x.get('macro_f1', 0))
    clf_data['extreme_f1'] = clf_data['performance'].apply(lambda x: x.get('extreme_f1', 0))
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('분류 모델 성능 비교', fontsize=16, fontweight='bold')
    
    # Accuracy 비교 (센터별)
    pivot_acc = clf_data.pivot_table(index='center', columns='model_name', values='accuracy', fill_value=0)
    pivot_acc.plot(kind='bar', ax=axes[0,0], color=plt.cm.Set3.colors)
    axes[0,0].set_title('센터별 정확도')
    axes[0,0].set_ylabel('Accuracy')
    axes[0,0].set_ylim(0, 1)
    axes[0,0].tick_params(axis='x', rotation=45)
    axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Macro F1 비교 (센터별)
    pivot_f1 = clf_data.pivot_table(index='center', columns='model_name', values='macro_f1', fill_value=0)
    pivot_f1.plot(kind='bar', ax=axes[0,1], color=plt.cm.Set2.colors)
    axes[0,1].set_title('센터별 Macro F1')
    axes[0,1].set_ylabel('Macro F1 Score')
    axes[0,1].set_ylim(0, 1)
    axes[0,1].tick_params(axis='x', rotation=45)
    axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 모델별 평균 정확도
    model_acc_avg = clf_data.groupby('model_name')['accuracy'].mean().sort_values(ascending=False)
    model_acc_avg.plot(kind='bar', ax=axes[1,0], color='lightgreen')
    axes[1,0].set_title('모델별 평균 정확도')
    axes[1,0].set_ylabel('평균 Accuracy')
    axes[1,0].set_ylim(0, 1)
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # 값 표시
    for i, v in enumerate(model_acc_avg.values):
        axes[1,0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # 극값 F1 비교
    pivot_extreme = clf_data.pivot_table(index='center', columns='model_name', values='extreme_f1', fill_value=0)
    pivot_extreme.plot(kind='bar', ax=axes[1,1], color=plt.cm.Pastel1.colors)
    axes[1,1].set_title('센터별 극값 F1 성능')
    axes[1,1].set_ylabel('Extreme F1 Score')
    axes[1,1].set_ylim(0, 1)
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"classification_performance_comparison_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  분류 성능 비교 저장: {os.path.basename(save_path)}")

def plot_model_performance_comparison(summary_df, results_dir, timestamp):
    """모델별 성능 비교 (전체 센터 통합)"""
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('모델별 성능 종합 비교', fontsize=16, fontweight='bold')
    
    # 회귀 모델들
    reg_data = summary_df[summary_df['task_type'] == 'regression'].copy()
    if len(reg_data) > 0:
        reg_data['r2'] = reg_data['performance'].apply(lambda x: x.get('r2', 0))
        reg_data['mae'] = reg_data['performance'].apply(lambda x: x.get('mae', 0))
        reg_data['mape'] = reg_data['performance'].apply(lambda x: x.get('mape', 0))
        
        # 모델별 R² 분포
        reg_models = reg_data['model_name'].unique()
        r2_data = [reg_data[reg_data['model_name'] == model]['r2'].values for model in reg_models]
        axes[0,0].boxplot(r2_data, labels=reg_models)
        axes[0,0].set_title('회귀 모델별 R² 분포')
        axes[0,0].set_ylabel('R² Score')
        axes[0,0].tick_params(axis='x', rotation=45)
        axes[0,0].grid(True, alpha=0.3)
        
        # 모델별 MAE 분포  
        mae_data = [reg_data[reg_data['model_name'] == model]['mae'].values for model in reg_models]
        axes[0,1].boxplot(mae_data, labels=reg_models)
        axes[0,1].set_title('회귀 모델별 MAE 분포')
        axes[0,1].set_ylabel('MAE')
        axes[0,1].tick_params(axis='x', rotation=45)
        axes[0,1].grid(True, alpha=0.3)
        
        # R² vs MAE 산점도
        axes[0,2].scatter(reg_data['r2'], reg_data['mae'], c=plt.cm.tab10.colors, s=100, alpha=0.7)
        for i, model in enumerate(reg_data['model_name']):
            axes[0,2].annotate(f"{reg_data.iloc[i]['center']}_{model[:3]}", 
                             (reg_data.iloc[i]['r2'], reg_data.iloc[i]['mae']),
                             xytext=(5, 5), textcoords='offset points', fontsize=8)
        axes[0,2].set_title('R² vs MAE')
        axes[0,2].set_xlabel('R² Score')
        axes[0,2].set_ylabel('MAE')
        axes[0,2].grid(True, alpha=0.3)
    
    # 분류 모델들
    clf_data = summary_df[summary_df['task_type'] == 'classification'].copy()
    if len(clf_data) > 0:
        clf_data['accuracy'] = clf_data['performance'].apply(lambda x: x.get('accuracy', 0))
        clf_data['macro_f1'] = clf_data['performance'].apply(lambda x: x.get('macro_f1', 0))
        clf_data['extreme_f1'] = clf_data['performance'].apply(lambda x: x.get('extreme_f1', 0))
        
        # 모델별 정확도 분포
        clf_models = clf_data['model_name'].unique()
        acc_data = [clf_data[clf_data['model_name'] == model]['accuracy'].values for model in clf_models]
        axes[1,0].boxplot(acc_data, labels=clf_models)
        axes[1,0].set_title('분류 모델별 정확도 분포')
        axes[1,0].set_ylabel('Accuracy')
        axes[1,0].tick_params(axis='x', rotation=45)
        axes[1,0].grid(True, alpha=0.3)
        
        # 모델별 F1 분포
        f1_data = [clf_data[clf_data['model_name'] == model]['macro_f1'].values for model in clf_models]
        axes[1,1].boxplot(f1_data, labels=clf_models)
        axes[1,1].set_title('분류 모델별 Macro F1 분포')
        axes[1,1].set_ylabel('Macro F1 Score')
        axes[1,1].tick_params(axis='x', rotation=45)
        axes[1,1].grid(True, alpha=0.3)
        
        # Accuracy vs F1 산점도
        axes[1,2].scatter(clf_data['accuracy'], clf_data['macro_f1'], c=plt.cm.tab10.colors, s=100, alpha=0.7)
        for i, model in enumerate(clf_data['model_name']):
            axes[1,2].annotate(f"{clf_data.iloc[i]['center']}_{model[:3]}", 
                             (clf_data.iloc[i]['accuracy'], clf_data.iloc[i]['macro_f1']),
                             xytext=(5, 5), textcoords='offset points', fontsize=8)
        axes[1,2].set_title('정확도 vs Macro F1')
        axes[1,2].set_xlabel('Accuracy')
        axes[1,2].set_ylabel('Macro F1 Score')
        axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"model_performance_comparison_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  모델 종합 비교 저장: {os.path.basename(save_path)}")

def plot_best_performance_summary(summary_df, results_dir, timestamp):
    """센터별 최고 성능 요약"""
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('센터별 최고 성능 모델 요약', fontsize=16, fontweight='bold')
    
    centers = summary_df['center'].unique()
    
    # 회귀 최고 성능
    reg_best = []
    reg_models = []
    for center in centers:
        center_reg = summary_df[(summary_df['center'] == center) & (summary_df['task_type'] == 'regression')]
        if len(center_reg) > 0:
            center_reg = center_reg.copy()
            center_reg['r2'] = center_reg['performance'].apply(lambda x: x.get('r2', 0))
            best_idx = center_reg['r2'].idxmax()
            reg_best.append(center_reg.loc[best_idx, 'r2'])
            reg_models.append(center_reg.loc[best_idx, 'model_name'][:8])  # 모델명 축약
        else:
            reg_best.append(0)
            reg_models.append('None')
    
    # 분류 최고 성능  
    clf_best = []
    clf_models = []
    for center in centers:
        center_clf = summary_df[(summary_df['center'] == center) & (summary_df['task_type'] == 'classification')]
        if len(center_clf) > 0:
            center_clf = center_clf.copy()
            center_clf['macro_f1'] = center_clf['performance'].apply(lambda x: x.get('macro_f1', 0))
            best_idx = center_clf['macro_f1'].idxmax()
            clf_best.append(center_clf.loc[best_idx, 'macro_f1'])
            clf_models.append(center_clf.loc[best_idx, 'model_name'][:8])  # 모델명 축약
        else:
            clf_best.append(0)
            clf_models.append('None')
    
    # 회귀 최고 성능 플롯
    bars1 = axes[0].bar(centers, reg_best, color='skyblue', alpha=0.7)
    axes[0].set_title('센터별 최고 회귀 성능 (R²)')
    axes[0].set_ylabel('R² Score')
    axes[0].set_ylim(0, 1)
    axes[0].tick_params(axis='x', rotation=45)
    
    # 값과 모델명 표시
    for i, (bar, score, model) in enumerate(zip(bars1, reg_best, reg_models)):
        axes[0].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                    f'{score:.3f}\n{model}', ha='center', va='bottom', fontsize=9)
    
    # 분류 최고 성능 플롯
    bars2 = axes[1].bar(centers, clf_best, color='lightcoral', alpha=0.7)
    axes[1].set_title('센터별 최고 분류 성능 (Macro F1)')
    axes[1].set_ylabel('Macro F1 Score')
    axes[1].set_ylim(0, 1)
    axes[1].tick_params(axis='x', rotation=45)
    
    # 값과 모델명 표시
    for i, (bar, score, model) in enumerate(zip(bars2, clf_best, clf_models)):
        axes[1].text(bar.get_x() + bar.get_width()/2, score + 0.01, 
                    f'{score:.3f}\n{model}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    save_path = os.path.join(results_dir, f"best_performance_summary_{timestamp}.png")
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"  최고 성능 요약 저장: {os.path.basename(save_path)}")

# ================================================================================================
# train_and_save_best_models 함수 수정 (성능 비교 그래프 생성 추가)
# ================================================================================================

def train_and_save_best_models_with_plots(cutoff_date='2025-05-20', save_dir='./trained_models', 
                                        results_dir='./results', create_plots=True):
    """모델 학습 + 성능 비교 그래프 생성"""
    
    # 기존 학습 과정 수행
    saved_models, training_summary = train_and_save_best_models(cutoff_date, save_dir, results_dir)
    
    # 성능 비교 그래프 생성
    if create_plots and training_summary:
        create_performance_comparison_plots(training_summary, results_dir)
    
    return saved_models, training_summary

# ================================================================================================
# 7. 2단계: 새로운 데이터로 예측 파이프라인
# ================================================================================================
def predict_with_saved_models(new_data_path, models_dir='./trained_models', 
                             results_dir='./results', historical_data_for_features=None):
    """저장된 모델들로 새로운 데이터 예측"""
    print("="*80)
    print("2단계: 새로운 데이터 예측")
    print("="*80)
    
    # 새로운 데이터 로드
    if isinstance(new_data_path, str):
        new_data = pd.read_csv(new_data_path, encoding='utf-8-sig')
        print(f"새 데이터 로드: {new_data_path} ({len(new_data)}행)")
    else:
        new_data = new_data_path.copy()
        print(f"새 데이터 제공됨: {len(new_data)}행")
    
    # 저장된 모델 목록 확인
    saved_models = load_all_saved_models(models_dir)
    
    if not saved_models:
        print("저장된 모델이 없습니다. 먼저 train_and_save_best_models()를 실행하세요.")
        return None
    
    all_predictions = []
    
    # 센터별 예측
    for center_name in saved_models.keys():
        print(f"\n[{center_name.upper()} 센터 예측]")
        
        # 해당 센터 데이터 필터링 (현재는 전체 데이터 사용)
        center_data = new_data.copy()
        
        if len(center_data) == 0:
            print(f"  해당 센터 데이터 없음")
            continue
        
        print(f"  예측 데이터: {len(center_data)}행")
        
        # 피처 생성
        if historical_data_for_features is not None:
            processed_data = make_features_for_prediction(
                historical_data_for_features[center_name], center_data
            )
        else:
            processed_data = make_simple_features(center_data)
        
        # 센터별 모델로 예측
        center_predictions = predict_for_center(center_name, processed_data, saved_models[center_name])
        all_predictions.extend(center_predictions)
    
    # 결과 정리
    if all_predictions:
        results_df = pd.DataFrame(all_predictions)
        results_df = results_df.sort_values(['center', 'date', 'task_type'])
        
        # 결과 저장 디렉토리 생성
        os.makedirs(results_dir, exist_ok=True)
        
        # 결과 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = os.path.join(results_dir, f"predictions_{timestamp}.csv")
        results_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        
        print(f"\n={'='*60}")
        print(f"예측 완료!")
        print(f"총 예측 건수: {len(results_df)}")
        print(f"결과 저장: {output_path}")
        print(f"={'='*60}")
        
        # 예측 결과 요약 출력
        print_prediction_summary(results_df)
        
        return results_df
    
    else:
        print("예측 결과가 없습니다.")
        return None

def load_all_saved_models(models_dir):
    """저장된 모든 모델 로드"""
    saved_models = {}
    
    if not os.path.exists(models_dir):
        return saved_models
    
    model_files = [f for f in os.listdir(models_dir) if f.endswith('.pkl') and f != 'training_summary.csv']
    
    for model_file in model_files:
        try:
            model_path = os.path.join(models_dir, model_file)
            
            with open(model_path, 'rb') as f:
                model_data = pickle.load(f)
            
            center_name = model_data['center_name']
            task_type = model_data['task_type']
            
            if center_name not in saved_models:
                saved_models[center_name] = {}
            
            saved_models[center_name][task_type] = model_data
            
            print(f"모델 로드: {center_name} {task_type} ({model_data['model_name']})")
            
        except Exception as e:
            print(f"모델 로드 실패 ({model_file}): {e}")
    
    return saved_models

def predict_for_center(center_name, processed_data, center_models):
    """센터별 모델로 예측"""
    predictions = []
    
    for task_type, model_data in center_models.items():
        try:
            pipeline = model_data['pipeline']
            feature_names = model_data['feature_names']
            model_name = model_data['model_name']
            target_col = model_data['target_column']
            
            # 예측 데이터 준비
            X_future = prepare_prediction_features(processed_data, feature_names)
            
            if X_future is None or len(X_future) == 0:
                continue
            
            # 예측 수행
            y_pred = pipeline.predict(X_future)
            
            print(f"  {task_type} 예측: {len(y_pred)}개")
            
            # 결과 저장
            for i in range(len(X_future)):
                pred_result = {
                    'date': processed_data.iloc[i]['날짜'],
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_name,
                    'target_column': target_col,
                    'predicted_value': int(y_pred[i]) if task_type == 'classification' else float(y_pred[i])
                }
                predictions.append(pred_result)
                
        except Exception as e:
            print(f"  {task_type} 예측 실패: {e}")
    
    return predictions

def print_prediction_summary(results_df):
    """예측 결과 요약 출력"""
    print(f"\n--- 예측 결과 요약 ---")
    
    for center in results_df['center'].unique():
        center_data = results_df[results_df['center'] == center]
        
        print(f"\n{center.upper()} 센터:")
        
        reg_data = center_data[center_data['task_type'] == 'regression']
        clf_data = center_data[center_data['task_type'] == 'classification']
        
        if len(reg_data) > 0:
            print(f"  회귀 예측: {len(reg_data)}개")
            print(f"  평균 예측값: {reg_data['predicted_value'].mean():.1f}")
            print(f"  예측 범위: {reg_data['predicted_value'].min():.1f} ~ {reg_data['predicted_value'].max():.1f}")
        
        if len(clf_data) > 0:
            print(f"  분류 예측: {len(clf_data)}개")
            grade_dist = clf_data['predicted_value'].value_counts().sort_index()
            print(f"  예측 등급 분포: {dict(grade_dist)}")
            
# ================================================================================================
# SHAP 분석 추가 코드 (버전13 + 성능비교 코드 뒤에 추가)
# ================================================================================================

def train_and_save_single_model_with_shap(center_name, train_data, best_result, task_type, save_dir, results_dir):
    """개별 모델 학습 및 저장 + SHAP 분석 포함"""
    try:
        model_name = best_result['model']
        split_method = best_result['split_method']
        target_col = "합계_1일후" if task_type == "regression" else "등급_1일후"
        
        # 전체 데이터로 학습
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=task_type, 
            test_size=0.2, split_method=split_method  # SHAP용으로 테스트 데이터 확보
        )
        
        # 전체 학습용 데이터 준비
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        
        # 모델 구축 및 학습
        if task_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, task_type)
        pipeline.fit(X_all, y_all)
        
        # 모델 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{center_name}_{model_name}_{task_type}_{timestamp}.pkl"
        model_path = os.path.join(save_dir, filename)
        
        model_data = {
            'pipeline': pipeline,
            'feature_names': feature_names,
            'model_name': model_name,
            'center_name': center_name,
            'task_type': task_type,
            'performance': dict(best_result),
            'split_method': split_method,
            'training_date': datetime.now().isoformat(),
            'target_column': target_col
        }
        
        with open(model_path, 'wb') as f:
            pickle.dump(model_data, f)
        
        print(f"    저장됨: {filename}")
        
        # SHAP 분석을 위한 별도 파이프라인 학습 (테스트 데이터용)
        analysis_pipeline = make_pipeline_unified(model, model_name, task_type)
        analysis_pipeline.fit(X_train, y_train)
        
        # Feature Importance 분석 및 저장
        try:
            importance_df = extract_feature_importance(analysis_pipeline, model_name, feature_names)
            if importance_df is not None:
                print(f"    Top 5 피처: {', '.join(importance_df.head(5)['feature'].tolist())}")
                
                # Feature Importance 시각화 저장
                fig = plot_feature_importance(importance_df, f"{center_name}_{model_name}")
                if fig:
                    img_path = os.path.join(results_dir, f"feature_importance_{center_name}_{model_name}_{task_type}_{timestamp}.png")
                    fig.savefig(img_path, dpi=300, bbox_inches='tight')
                    plt.close(fig)
                    print(f"    Feature Importance 저장: {os.path.basename(img_path)}")
        except Exception as e:
            print(f"    Feature Importance 분석 실패: {e}")
        
        # SHAP 분석 및 저장
        if HAS_SHAP:
            try:
                print(f"    SHAP 분석 시작...")
                shap_result = analyze_model_with_shap(analysis_pipeline, X_test, feature_names, model_name, max_samples=50)
                
                if shap_result:
                    shap_values, X_processed, explainer = shap_result
                    
                    # SHAP 시각화
                    shap_figs = plot_shap_summary(shap_values, X_processed, feature_names, f"{center_name}_{model_name}")
                    
                    for i, fig in enumerate(shap_figs):
                        suffix = "bar" if i == 0 else "beeswarm"
                        shap_img_path = os.path.join(results_dir, f"shap_{suffix}_{center_name}_{model_name}_{task_type}_{timestamp}.png")
                        fig.savefig(shap_img_path, dpi=300, bbox_inches='tight')
                        plt.close(fig)
                        print(f"    SHAP {suffix} 저장: {os.path.basename(shap_img_path)}")
                    
                    # SHAP 값 요약 저장 (CSV)
                    if isinstance(shap_values, np.ndarray):
                        mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
                        shap_summary_df = pd.DataFrame({
                            'feature': feature_names,
                            'mean_abs_shap': mean_abs_shap
                        }).sort_values('mean_abs_shap', ascending=False)
                        
                        shap_csv_path = os.path.join(results_dir, f"shap_summary_{center_name}_{model_name}_{task_type}_{timestamp}.csv")
                        shap_summary_df.to_csv(shap_csv_path, index=False, encoding='utf-8-sig')
                        print(f"    SHAP 요약 저장: {os.path.basename(shap_csv_path)}")
                
            except Exception as e:
                print(f"    SHAP 분석 실패: {e}")
        else:
            print(f"    SHAP 라이브러리 없음 - SHAP 분석 건너뜀")
        
        return {
            'model_name': model_name,
            'performance': dict(best_result),
            'saved_path': model_path,
            'feature_names': feature_names,
            'target_column': target_col
        }
        
    except Exception as e:
        print(f"    모델 저장 실패: {e}")
        return None

def select_and_save_best_models_with_shap(center_name, train_data, training_results, save_dir, results_dir):
    """센터별 최고 성능 모델 선택 및 저장 + SHAP 분석"""
    results_df = pd.DataFrame(training_results)
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) == 0:
        return None
    
    saved_models = {}
    
    # 회귀 최고 성능 모델
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"  최고 회귀 모델: {best_reg['model']} (R²={best_reg['r2']:.3f})")
        
        # 모델 학습 및 저장 (SHAP 포함)
        model_data = train_and_save_single_model_with_shap(
            center_name, train_data, best_reg, 'regression', save_dir, results_dir
        )
        
        if model_data:
            saved_models['regression'] = model_data
    
    # 분류 최고 성능 모델
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"  최고 분류 모델: {best_clf['model']} (F1={best_clf['macro_f1']:.3f})")
        
        # 모델 학습 및 저장 (SHAP 포함)
        model_data = train_and_save_single_model_with_shap(
            center_name, train_data, best_clf, 'classification', save_dir, results_dir
        )
        
        if model_data:
            saved_models['classification'] = model_data
    
    return saved_models

def train_and_save_best_models_with_plots_and_shap(cutoff_date='2025-05-20', save_dir='./trained_models', 
                                                  results_dir='./results', create_plots=True):
    """모델 학습 + SHAP 분석 + 성능 비교 그래프 생성"""
    print("="*80)
    print("1단계: 모델 학습 및 저장 (SHAP 분석 포함)")
    print("="*80)
    
    # 저장 디렉토리 생성
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)
    
    # 데이터 로드
    centers_data = load_original_data()
    cutoff = pd.to_datetime(cutoff_date)
    
    saved_models = {}
    training_summary = []
    
    for center_name, df_raw in centers_data.items():
        print(f"\n[{center_name.upper()} 센터 모델 학습]")
        
        # 학습 데이터 준비
        df_raw['날짜'] = pd.to_datetime(df_raw['날짜'])
        raw_train_data = df_raw[df_raw['날짜'] <= cutoff].copy()
        
        if len(raw_train_data) < 50:
            print(f"  학습 데이터 부족")
            continue
        
        # 피처 엔지니어링
        train_data = make_features(raw_train_data, cutoff_date=cutoff_date)
        print(f"  학습 데이터: {len(train_data)}행")
        
        # 모델 평가
        training_results = comprehensive_evaluation_comparison(center_name, train_data)
        
        # 최고 성능 모델 선택 및 학습 (SHAP 포함)
        best_models = select_and_save_best_models_with_shap(center_name, train_data, training_results, save_dir, results_dir)
        
        if best_models:
            saved_models[center_name] = best_models
            
            # 학습 요약 정보 저장
            for task_type, model_info in best_models.items():
                training_summary.append({
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_info['model_name'],
                    'performance': model_info['performance'],
                    'saved_path': model_info['saved_path'],
                    'training_date': datetime.now().isoformat()
                })
    
    # 학습 요약 저장
    summary_path = os.path.join(save_dir, 'training_summary.csv')
    summary_df = pd.DataFrame(training_summary)
    summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
    
    print(f"\n={'='*60}")
    print(f"모델 학습 완료!")
    print(f"저장 위치: {save_dir}")
    print(f"학습된 모델: {len(training_summary)}개")
    print(f"학습 요약: {summary_path}")
    print(f"={'='*60}")
    
    # 성능 비교 그래프 생성
    if create_plots and training_summary:
        create_performance_comparison_plots(training_summary, results_dir)
    
    return saved_models, training_summary

# ================================================================================================
# 최종 사용법
# ================================================================================================
def final_example_usage():
    """SHAP 포함 최종 사용 예시"""
    print("="*80)
    print("SHAP 분석 포함 완전한 파이프라인 사용법")
    print("="*80)
    
    print("\n1. SHAP + 성능비교 그래프 포함 학습:")
    print("saved_models, summary = train_and_save_best_models_with_plots_and_shap()")
    
    print("\n2. 예측:")
    print("results = predict_with_saved_models('새_데이터.csv')")
    
    print("\n최종 산출물 (총 29개 파일):")
    print("="*40)
    print("./trained_models/ (9개)")
    print("├── 모델 파일 8개 (.pkl)")
    print("└── training_summary.csv")
    
    print("\n./results/ (20개)")
    print("├── Feature Importance 이미지 8개")
    print("├── SHAP Bar 이미지 8개") 
    print("├── SHAP Beeswarm 이미지 8개")
    print("├── SHAP 요약 CSV 8개")
    print("└── 성능 비교 그래프 4개")
    
    print("\n예측 시 추가:")
    print("└── predictions_*.csv")
    
    print("\n주의사항:")
    print("- SHAP 라이브러리 필요: pip install shap")
    print("- SHAP 분석으로 시간이 더 오래 걸림")
    print("- 메모리 사용량 증가")

if __name__ == "__main__":
    final_example_usage()

# ================================================================================================
# 8. 사용 가이드 및 실행 예시
# ================================================================================================
def example_usage():
    """사용 예시 가이드"""
    print("="*80)
    print("하수처리장 예측 파이프라인 사용 가이드")
    print("="*80)
    
    print("\n1단계: 모델 학습 및 저장 (한 번만 실행)")
    print("="*50)
    print("saved_models, summary = train_and_save_best_models(cutoff_date='2025-05-20')")
    print()
    print("생성되는 파일:")
    print("- ./trained_models/ 폴더")
    print("  ├── nanji_RandomForest_Reg_regression_20250101_123456.pkl")
    print("  ├── nanji_LogisticRegression_Clf_classification_20250101_123456.pkl")
    print("  ├── ... (최대 8개 모델)")
    print("  ├── training_summary.csv")
    print("  └── feature_importance_*.png (시각화 이미지들)")
    
    print("\n2단계: 새로운 데이터로 예측 (반복 실행 가능)")
    print("="*50)
    print("# CSV 파일로 예측")
    print("results = predict_with_saved_models('새로운_데이터.csv')")
    print()
    print("# DataFrame으로 예측")
    print("new_df = pd.read_csv('새로운_데이터.csv')")
    print("results = predict_with_saved_models(new_df)")
    print()
    print("# 과거 데이터와 함께 예측 (더 정확한 시차 변수)")
    print("historical_data = load_original_data()")
    print("results = predict_with_saved_models('새로운_데이터.csv',")
    print("                                   historical_data_for_features=historical_data)")
    
    print("\n새로운 데이터 형식 예시:")
    print("="*30)
    print("날짜,일_평균기온(°C),일_일강수량(mm),평균습도(%),계절,불쾌지수등급")
    print("2025-06-01,25.3,0,65,여름,약간 불쾌")
    print("2025-06-02,26.1,2.5,70,여름,불쾌")
    print("2025-06-03,24.8,15.2,75,여름,매우 불쾌")
    
    print("\n예측 결과 형식:")
    print("="*20)
    print("date,center,task_type,model_name,predicted_value")
    print("2025-06-01,nanji,regression,RandomForest_Reg,1250.5")
    print("2025-06-01,nanji,classification,LogisticRegression_Clf,2")
    
    print("\n주의사항:")
    print("="*15)
    print("- 모델 학습은 한 번만 실행하면 됩니다")
    print("- 예측은 새로운 데이터가 있을 때마다 실행 가능")
    print("- 데이터 경로가 올바른지 확인하세요")
    print("- 새 데이터의 컬럼 구조는 기존 학습 데이터와 유사해야 합니다")

def updated_example_usage():
    """업데이트된 사용 예시"""
    print("="*80)
    print("성능 비교 그래프 포함 파이프라인 사용법")
    print("="*80)
    
    print("\n1. 성능 비교 그래프 포함 학습:")
    print("saved_models, summary = train_and_save_best_models_with_plots()")
    
    print("\n2. 성능 비교 그래프만 별도 생성:")
    print("# 학습 요약 데이터가 있는 경우")
    print("create_performance_comparison_plots(training_summary, results_dir='./results')")
    
    print("\n생성되는 성능 비교 그래프:")
    print("- regression_performance_comparison_*.png")
    print("- classification_performance_comparison_*.png") 
    print("- model_performance_comparison_*.png")
    print("- best_performance_summary_*.png")
    
    print("\n그래프 내용:")
    print("1. 회귀: 센터별/모델별 R², MAE 비교")
    print("2. 분류: 센터별/모델별 정확도, F1 비교")
    print("3. 모델 종합: 박스플롯, 산점도 분석")
    print("4. 최고 성능: 센터별 베스트 모델 요약")

if __name__ == "__main__":
    updated_example_usage()


# ================================================================================================
# 실행 준비 완료
# ================================================================================================
print("\n" + "="*80)
print("완전한 하수처리장 예측 파이프라인 준비 완료!")
print("="*80)
print("\n다음 명령어로 시작하세요:")
print("\n# 1단계: 모델 학습")
print("saved_models, summary = train_and_save_best_models()")
print("\n# 2단계: 예측")
print("results = predict_with_saved_models('새로운_데이터.csv')")
print("\n사용 가이드: example_usage()")
print("="*80)

하수처리장 예측 파이프라인 초기화 완료
XGBoost: ✓
LightGBM: ✓
CatBoost: ✓
SHAP: ✓
하수처리장 예측 파이프라인 사용 가이드

1단계: 모델 학습 및 저장 (한 번만 실행)
saved_models, summary = train_and_save_best_models(cutoff_date='2025-05-20')

생성되는 파일:
- ./trained_models/ 폴더
  ├── nanji_RandomForest_Reg_regression_20250101_123456.pkl
  ├── nanji_LogisticRegression_Clf_classification_20250101_123456.pkl
  ├── ... (최대 8개 모델)
  ├── training_summary.csv
  └── feature_importance_*.png (시각화 이미지들)

2단계: 새로운 데이터로 예측 (반복 실행 가능)
# CSV 파일로 예측
results = predict_with_saved_models('새로운_데이터.csv')

# DataFrame으로 예측
new_df = pd.read_csv('새로운_데이터.csv')
results = predict_with_saved_models(new_df)

# 과거 데이터와 함께 예측 (더 정확한 시차 변수)
historical_data = load_original_data()
results = predict_with_saved_models('새로운_데이터.csv',
                                   historical_data_for_features=historical_data)

새로운 데이터 형식 예시:
날짜,일_평균기온(°C),일_일강수량(mm),평균습도(%),계절,불쾌지수등급
2025-06-01,25.3,0,65,여름,약간 불쾌
2025-06-02,26.1,2.5,70,여름,불쾌
2025-06-03,24.8,15.2,75,여름,매우 불쾌

예측 결과 형식:
d

In [8]:
saved_models, summary = train_and_save_best_models()

1단계: 모델 학습 및 저장

[NANJI 센터 모델 학습]
  학습 데이터: 3058행

센터: nanji - Stratified vs 시계열 분할 비교
데이터 크기: 3058행, 44컬럼
등급 분포: {0.0: 458, 1.0: 1682, 2.0: 611, 3.0: 307}
클래스 불균형 비율: 5.5:1 (최대:1682, 최소:307)

분할 방법: TEMPORAL

--- 회귀 모델 평가 (temporal) ---
회귀용 데이터: 학습 2446행, 테스트 612행


회귀(temporal):  17%|█▋        | 1/6 [00:00<00:04,  1.24it/s]

  RandomForest_Reg  : R²=0.557, MAE=49542, MAPE=7.3%
  LinearRegression  : R²=0.513, MAE=62439, MAPE=9.7%


회귀(temporal):  50%|█████     | 3/6 [00:02<00:02,  1.21it/s]

  GradientBoosting_Reg: R²=0.395, MAE=58665, MAPE=9.0%


...

  XGBoost_Clf       : ACC=0.658, F1=0.547, 극값F1=0.488


분류(stratified):  83%|████████▎ | 5/6 [00:16<00:03,  3.80s/it]

  LightGBM_Clf      : ACC=0.647, F1=0.545, 극값F1=0.515


  CatBoost_Clf      : ACC=0.614, F1=0.559, 극값F1=0.577
  최고 회귀 모델: CatBoost_Reg (R²=0.515)


    저장됨: tancheon_CatBoost_Reg_regression_20250827_165921.pkl
    Top 5 피처: 세탁업, 체력단련장업, 강수량_7일_누적, 하천, 일_일강수량(mm)
    Feature Importance 저장: feature_importance_tancheon_CatBoost_Reg_regression_20250827_165921.png
  최고 분류 모델: CatBoost_Clf (F1=0.559)
    저장됨: tancheon_CatBoost_Clf_classification_20250827_165922.pkl
    Top 5 피처: 세탁업, 강수량_7일_누적, 생활인구, 월, 체력단련장업
    Feature Importance 저장: feature_importance_tancheon_CatBoost_Clf_classification_20250827_165922.png

모델 학습 완료!
저장 위치: ./trained_models
학습된 모델: 8개
학습 요약: ./trained_models/training_summary.csv


In [ ]:
# results = predict_with_saved_models('새_데이터.csv')